# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-e4c93df472-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series HIMYM explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "How I met your mother" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q147235` | How I met your mother        | node |
| `wd:Q23831` | The Office (US)        | node |



Also consider

```
wd:Q23831 ?p ?obj .
```

is the BGP to retrieve all **properties of The Office (US)**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should

1. Identify the BGP for tv series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for tv series

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/

In [3]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')

# EXPLORING The Office (US):
#  Increasing LIMIT 
#  Returning also the object

queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q23831 ?p ?o.
   # get the label
   ?p sc:name ?name.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('name', 'original broadcaster'), ('o', 'http://www.wikidata.org/entity/Q13974'), ('oName', 'NBC')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q21664088'), ('oName', 'two-part episode')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5398426'), ('oName', 'television series')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('name', 'distribution format'), ('o', 'http://www.wikidata.org/entity/Q723685'), ('oName', 'video on demand')]
[('p', 'h

In [4]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')
# wdt:P1113 ('number of episodes')

# EXPLORING The Office (US):
#  Increasing LIMIT 
#  Returning also the object
#  FILTERing number of episodes

queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   OPTIONAL {
       ?o sc:name ?oName
   }
   
   FILTER (?p not in (wdt:P1113))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID'), ('o', 'http://www.wikidata.org/entity/P348'), ('oName', 'software version identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID'), ('o', 'http://www.wikidata.org/entity/P4669'), ('oName', 'tabular software version')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID'), ('o', 'http://www.wikidata.org/entity/P747'), ('oName', 'has edition or translation')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID'), ('o', 'http://www.wikidata.org/entity/Q1002728'), ('oName', 'Fonsi Nieto')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID'), ('o', 'http://www.wikidata.org/entity/Q100326534'), ('oName', "Al Stewart's albums in chronological order")]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID'), ('o', 'http://www.wikidata.org/entit

In [5]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')
# wdt:P1113 ('number of episodes')
# wdt:P1258 ('Rotten Tomatoes ID')

# EXPLORING The Office (US):
#  Increasing LIMIT 
#  Returning also the object
#  FILTERing number of episodes, 'Rotten Tomatoes ID'

queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   OPTIONAL {
       ?o sc:name ?oName
   }
   
   FILTER (?p not in (wdt:P1113, wdt:P1258))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID'), ('o', 'http://www.wikidata.org/entity/P348'), ('oName', 'software version identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID'), ('o', 'http://www.wikidata.org/entity/P4669'), ('oName', 'tabular software version')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID'), ('o', 'http://www.wikidata.org/entity/P747'), ('oName', 'has edition or translation')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID'), ('o', 'http://www.wikidata.org/entity/Q1002728'), ('oName', 'Fonsi Nieto')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID'), ('o', 'http://www.wikidata.org/entity/Q100326534'), ('oName', "Al Stewart's albums in chronological order")]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID'), ('o', 'http://www.wikidata.org/entit

In [6]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')
# wdt:P1113 ('number of episodes')
# wdt:P1258 ('Rotten Tomatoes ID')

# EXPLORING The Office (US):
#  Increasing LIMIT 

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   
   # get the label
   ?p sc:name ?name.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/

#### Found 'television series' but not 'tv series'

In [7]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')
# wdt:P1113 ('number of episodes')
# wdt:P1258 ('Rotten Tomatoes ID')
# wdt:P31 (instance of')

# EXPLORING The Office (US):
#  Increasing LIMIT 
#  Getting the instance of 

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q23831 wdt:P31 ?o.
   
   # get the label
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5398426'), ('oName', 'television series')]
1


In [8]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')
# wdt:P1113 ('number of episodes')
# wdt:P1258 ('Rotten Tomatoes ID')
# wdt:P31 (instance of')

# EXPLORING The Office (US) - BACKWARD

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName 
WHERE {
   # bind something
   ?s ?p wd:Q23831.
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?s sc:name ?sName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q50379836'), ('sName', 'Classy Christmas (part 1)'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('s', 'http://www.wikidata.org/entity/Q50379837'), ('sName', 'Classy Christmas (part 2)'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('s', 'http://www.wikidata.org/entity/Q5099551'), ('sName', 'China'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('s', 'http://www.wikidata.org/entity/Q5128465'), ('sName', 'Classy Christmas'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('s', 'http://www.wikidata.org/entity/Q5178024'), ('sName', 'Couples Discount'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('s', 'http://www.wikidata.org/entity/Q6927074'), ('sName', 'Moving On'), ('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the seri

In [9]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')
# wdt:P1113 ('number of episodes')
# wdt:P1258 ('Rotten Tomatoes ID')
# wdt:P31 (instance of')

# EXPLORING The Office (US) - BACKWARD
#   FILTERing 

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName 
WHERE {
   # bind something
   ?s ?p wd:Q23831.
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?s sc:name ?sName
   }
   
   # wdt:179 ('part of the series'), wdt:P1441 ('present in work')
   FILTER (?p not in (wdt:P179, wdt:P1441))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2413177'), ('sName', 'The Office – A XXX Parody'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q23829'), ('sName', 'The Office'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q8849068'), ('sName', 'Category:The Office (American TV series)'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q7994037'), ('sName', 'Whistleblower'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q5001791'), ('sName', 'Business School'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q23829'), ('sName', 'The Office'), ('p', 'http://www.wikidata.org/prop/direct/P4969'), ('pName', 'derivative work')]
[('s', 'http://ww

In [10]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)')
# wdt:P1113 ('number of episodes')
# wdt:P1258 ('Rotten Tomatoes ID')
# wdt:P31 (instance of')
# wdt:P136 ('genre')

# EXPLORING The Office (US):
#  Increasing LIMIT 
#  Getting the genre

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q23831 wdt:P136 ?o .
   
   # get the label
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q21188110'), ('oName', 'American television sitcom')]
[('o', 'http://www.wikidata.org/entity/Q459435'), ('oName', 'mockumentary')]
2


### Final query for this task

#### By assuming tv == television

In [7]:
# Identify the BGP for tv series
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q23831 wdt:P31 ?o.
   
   # get the label
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5398426'), ('oName', 'television series')]
1


## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

#### Two different interpretation
- we are talking about only one tv series: 
    1) The Office (US): is the input for the Task 1
    2) How I met your mother: is main topic of the notebook
- we are talking about all the tv series in the db, the the sol would be:
    1) Find all tv series
    2) Find the episodes

In [24]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# 1) finding all tv series

# BACKWARD search from wd:Q5398426 ('television series') with wdt:P31 (instance of')

queryString = """
SELECT DISTINCT ?series ?seriesName
WHERE {
   # bind something
   ?series wdt:P31 wd:Q5398426.
   
   # get the label
   OPTIONAL {
       ?series sc:name ?seriesName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('series', 'http://www.wikidata.org/entity/Q1154217'), ('seriesName', 'Ironfist Chinmi')]
[('series', 'http://www.wikidata.org/entity/Q691528'), ('seriesName', 'Wallace and Gromit')]
[('series', 'http://www.wikidata.org/entity/Q8193809'), ('seriesName', 'Shomuni')]
[('series', 'http://www.wikidata.org/entity/Q33323'), ('seriesName', 'Everybody Hates Chris')]
[('series', 'http://www.wikidata.org/entity/Q500769'), ('seriesName', 'Freaks and Geeks')]
[('series', 'http://www.wikidata.org/entity/Q8350955'), ('seriesName', 'Cordel Encantado')]
[('series', 'http://www.wikidata.org/entity/Q1145054'), ('seriesName', 'The NBC Mystery Movie')]
[('series', 'http://www.wikidata.org/entity/Q344081'), ('seriesName', 'Recess')]
[('series', 'http://www.wikidata.org/entity/Q13647845'), ('seriesName', 'Grijpstra & De Gier')]
[('series', 'http://www.wikidata.org/entity/Q708661'), ('seriesName', 'Lindenstraße')]
[('series', 'http://www.wikidata.org/entity/Q15099450'), ('seriesName', 'Prime Ministe

#### Two different interpretation: starting with the first one
High chance to be the first one because
1) there are to many series
2) in the parentheses they are requesting only the season IRI, label and #episodes.

##### Finding episodes of The Office (US)

In [25]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P527 ('has part')--> wd:Q3468601 ('The Office, season 2')
#                               --wdt:P527 ('has part')--> wd:Q3468797 ('The Office, season 3')
#                               --wdt:P527 ('has part')--> wd:Q3468906 ('The Office, season 4')
#                               --wdt:P527 ('has part')--> wd:Q3468990 ('The Office, season 5')

# 1) finding the season of The Office

# EXPLORING The Office

queryString = """
SELECT DISTINCT ?part ?partName
WHERE {
   # bind something
   wd:Q23831 wdt:P527 ?part.
   
   # get the label
   OPTIONAL {
       ?part sc:name ?partName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('part', 'http://www.wikidata.org/entity/Q3465812'), ('partName', 'The Office, season 1')]
[('part', 'http://www.wikidata.org/entity/Q3468601'), ('partName', 'The Office, season 2')]
[('part', 'http://www.wikidata.org/entity/Q3468797'), ('partName', 'The Office, season 3')]
[('part', 'http://www.wikidata.org/entity/Q3468906'), ('partName', 'The Office, season 4')]
[('part', 'http://www.wikidata.org/entity/Q3468990'), ('partName', 'The Office, season 5')]
[('part', 'http://www.wikidata.org/entity/Q3730253'), ('partName', 'The Office, season 9')]
[('part', 'http://www.wikidata.org/entity/Q3730255'), ('partName', 'The Office, season 8')]
[('part', 'http://www.wikidata.org/entity/Q3730261'), ('partName', 'The Office, season 6')]
[('part', 'http://www.wikidata.org/entity/Q3730263'), ('partName', 'The Office, season 7')]
9


In [26]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P527 ('has part')--> wd:Q3468601 ('The Office, season 2')
#                               --wdt:P527 ('has part')--> wd:Q3468797 ('The Office, season 3')
#                               --wdt:P527 ('has part')--> wd:Q3468906 ('The Office, season 4')
#                               --wdt:P527 ('has part')--> wd:Q3468990 ('The Office, season 5')

# 1) finding the season of The Office

# EXPLORING The Office, season 1

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q3465812 ?p ?o.
   
   # get the label
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('oName', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P2747'), ('pName', 'Filmiroda rating'), ('o', 'http://www.wikidata.org/entity/Q23830577'), ('oName', 'Category II')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3464665'), ('oName', 'television series season')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q7713172'), ('oName', 'The Alliance')]
[('p', 'http://

In [29]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')
#                               --wdt:P527 ('has part')--> wd:Q3468601 ('The Office, season 2')
#                               --wdt:P527 ('has part')--> wd:Q3468797 ('The Office, season 3')
#                               --wdt:P527 ('has part')--> wd:Q3468906 ('The Office, season 4')
#                               --wdt:P527 ('has part')--> wd:Q3468990 ('The Office, season 5')

# 1) finding the season of The Office

# EXPLORING The Office of all seasons

queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   wd:Q23831 wdt:P527 ?season.
   ?season wdt:P1113 ?episodes.
   
   # get the label
   ?season sc:name ?seasonName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q3465812'), ('seasonName', 'The Office, season 1'), ('episodes', '6')]
[('season', 'http://www.wikidata.org/entity/Q3468601'), ('seasonName', 'The Office, season 2'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q3468797'), ('seasonName', 'The Office, season 3'), ('episodes', '25')]
[('season', 'http://www.wikidata.org/entity/Q3468906'), ('seasonName', 'The Office, season 4'), ('episodes', '19')]
[('season', 'http://www.wikidata.org/entity/Q3468990'), ('seasonName', 'The Office, season 5'), ('episodes', '28')]
[('season', 'http://www.wikidata.org/entity/Q3730253'), ('seasonName', 'The Office, season 9'), ('episodes', '25')]
[('season', 'http://www.wikidata.org/entity/Q3730255'), ('seasonName', 'The Office, season 8'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q3730261'), ('seasonName', 'The Office, season 6'), ('episodes', '26')]
[('season', 'http://www.wikidata.org/entity/Q3730263'), ('seasonN

In [30]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')
#                               --wdt:P527 ('has part')--> wd:Q3468601 ('The Office, season 2')
#                               --wdt:P527 ('has part')--> wd:Q3468797 ('The Office, season 3')
#                               --wdt:P527 ('has part')--> wd:Q3468906 ('The Office, season 4')
#                               --wdt:P527 ('has part')--> wd:Q3468990 ('The Office, season 5')

# 1) finding the season of The Office

# EXPLORING The Office of all seasons
# ORDERING

queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   wd:Q23831 wdt:P527 ?season.
   ?season wdt:P1113 ?episodes.
   
   # get the label
   ?season sc:name ?seasonName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q3465812'), ('seasonName', 'The Office, season 1'), ('episodes', '6')]
[('season', 'http://www.wikidata.org/entity/Q3468601'), ('seasonName', 'The Office, season 2'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q3468797'), ('seasonName', 'The Office, season 3'), ('episodes', '25')]
[('season', 'http://www.wikidata.org/entity/Q3468906'), ('seasonName', 'The Office, season 4'), ('episodes', '19')]
[('season', 'http://www.wikidata.org/entity/Q3468990'), ('seasonName', 'The Office, season 5'), ('episodes', '28')]
[('season', 'http://www.wikidata.org/entity/Q3730261'), ('seasonName', 'The Office, season 6'), ('episodes', '26')]
[('season', 'http://www.wikidata.org/entity/Q3730263'), ('seasonName', 'The Office, season 7'), ('episodes', '26')]
[('season', 'http://www.wikidata.org/entity/Q3730255'), ('seasonName', 'The Office, season 8'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q3730253'), ('seasonN

##### SAME FOR wd:Q147235 How I met your mother

In [31]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')
#                               --wdt:P527 ('has part')--> wd:Q3468601 ('The Office, season 2')
#                               --wdt:P527 ('has part')--> wd:Q3468797 ('The Office, season 3')
#                               --wdt:P527 ('has part')--> wd:Q3468906 ('The Office, season 4')
#                               --wdt:P527 ('has part')--> wd:Q3468990 ('The Office, season 5')

# wd:Q147235 How I met your mother
# 1) finding the season of The Office

# EXPLORING 'How I met your mother' all seasons with episoded
# ORDERING

queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   wd:Q147235 wdt:P527 ?season.
   ?season wdt:P1113 ?episodes.
   
   # get the label
   ?season sc:name ?seasonName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2715578'), ('seasonName', 'How I Met Your Mother, season 1'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q3468515'), ('seasonName', 'How I Met Your Mother, season 2'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q2555117'), ('seasonName', 'How I Met Your Mother, season 3'), ('episodes', '20')]
[('season', 'http://www.wikidata.org/entity/Q2567330'), ('seasonName', 'How I Met Your Mother, season 4'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q582332'), ('seasonName', 'How I Met Your Mother, season 5'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2472427'), ('seasonName', 'How I Met Your Mother, season 7'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q338715'), ('seasonName', 'How I Met Your Mother, season 8')

##### Putting togheter: wd:Q23831 'The Office (US)' and wd:Q147235 How I met your mother 

In [33]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')
#                               --wdt:P527 ('has part')--> wd:Q3468601 ('The Office, season 2')
#                               --wdt:P527 ('has part')--> wd:Q3468797 ('The Office, season 3')
#                               --wdt:P527 ('has part')--> wd:Q3468906 ('The Office, season 4')
#                               --wdt:P527 ('has part')--> wd:Q3468990 ('The Office, season 5')

# 1) finding the season of The Office

# EXPLORING The Office of all seasons
# ORDERING

queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P527 ?season.
   ?season wdt:P1113 ?episodes.
   
   # get the label
   ?season sc:name ?seasonName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2715578'), ('seasonName', 'How I Met Your Mother, season 1'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q3468515'), ('seasonName', 'How I Met Your Mother, season 2'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q2555117'), ('seasonName', 'How I Met Your Mother, season 3'), ('episodes', '20')]
[('season', 'http://www.wikidata.org/entity/Q2567330'), ('seasonName', 'How I Met Your Mother, season 4'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q582332'), ('seasonName', 'How I Met Your Mother, season 5'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2472427'), ('seasonName', 'How I Met Your Mother, season 7'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q338715'), ('seasonName', 'How I Met Your Mother, season 8')

#### For all tv series

In [34]:
# Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).
# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')

# 1) finding all tv series

queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   ?tvSeries wdt:P31 wd:Q5398426;
             wdt:P527 ?season.
   ?season wdt:P1113 ?episodes.
   
   # get the label
   ?season sc:name ?seasonName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q18816757'), ('seasonName', '12 Monkeys, season 1'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q25093994'), ('seasonName', '12 Monkeys, season 2'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q46555732'), ('seasonName', '12 Monkeys, season 3'), ('episodes', '10')]
[('season', 'http://www.wikidata.org/entity/Q46555698'), ('seasonName', '12 Monkeys, season 4'), ('episodes', '10')]
[('season', 'http://www.wikidata.org/entity/Q29360820'), ('seasonName', '13 Reasons Why, season 1'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q53679951'), ('seasonName', '13 Reasons Why, season 2'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q66712436'), ('seasonName', '13 Reasons Why, season 3'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q96106882'), ('seasonName', '13 Reasons Why, season 4'), ('episodes', '10')]
[('season', 'http://www.wikidata.org/ent

### Final query for this task

#### Solution A: 'The Office (US)' and 'How I met your mother' only tv series 

In [35]:
# Return the number of seasons and episodes per season of the tv series 
#   (the result set must be triples of season IRI, label and #episodes).

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')

queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P527 ?season.  #--wdt:P527 ('has part')-->
   ?season wdt:P1113 ?episodes. #--wdt:P1113 ('number of episodes')-->
   
   # get the label
   ?season sc:name ?seasonName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2715578'), ('seasonName', 'How I Met Your Mother, season 1'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q3468515'), ('seasonName', 'How I Met Your Mother, season 2'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q2555117'), ('seasonName', 'How I Met Your Mother, season 3'), ('episodes', '20')]
[('season', 'http://www.wikidata.org/entity/Q2567330'), ('seasonName', 'How I Met Your Mother, season 4'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q582332'), ('seasonName', 'How I Met Your Mother, season 5'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2472427'), ('seasonName', 'How I Met Your Mother, season 7'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q338715'), ('seasonName', 'How I Met Your Mother, season 8')

#### Solution B: all the tv series 

In [36]:
# Return the number of seasons and episodes per season of the tv series 
#   (the result set must be triples of season IRI, label and #episodes).

# wd:Q23831 ('The Office (US)') --wdt:P31 (instance of')--> wd:Q5398426 ('television series')


queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   ?tvSeries wdt:P31 wd:Q5398426; #--wdt:P31 ('instance of')-->
             wdt:P527 ?season.    #--wdt:P527 ('has part')-->
   ?season wdt:P1113 ?episodes.   #--wdt:P1113 ('number of episodes')-->
   
   # get the label
   ?season sc:name ?seasonName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q18816757'), ('seasonName', '12 Monkeys, season 1'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q25093994'), ('seasonName', '12 Monkeys, season 2'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q46555732'), ('seasonName', '12 Monkeys, season 3'), ('episodes', '10')]
[('season', 'http://www.wikidata.org/entity/Q46555698'), ('seasonName', '12 Monkeys, season 4'), ('episodes', '10')]
[('season', 'http://www.wikidata.org/entity/Q29360820'), ('seasonName', '13 Reasons Why, season 1'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q53679951'), ('seasonName', '13 Reasons Why, season 2'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q66712436'), ('seasonName', '13 Reasons Why, season 3'), ('episodes', '13')]
[('season', 'http://www.wikidata.org/entity/Q96106882'), ('seasonName', '13 Reasons Why, season 4'), ('episodes', '10')]
[('season', 'http://www.wikidata.org/ent

## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

In [37]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)')
# wdt:P161 ('cast member')

# finding the cast members 

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q23831 wdt:P161 ?o.
   
   # get the label
   OPTIONAL{
       ?o sc:name ?oName.
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1050211'), ('oName', 'Leslie David Baker')]
[('o', 'http://www.wikidata.org/entity/Q1139248'), ('oName', 'Oscar Nunez')]
[('o', 'http://www.wikidata.org/entity/Q216221'), ('oName', 'Steve Carell')]
[('o', 'http://www.wikidata.org/entity/Q2238008'), ('oName', 'Creed Bratton')]
[('o', 'http://www.wikidata.org/entity/Q231203'), ('oName', 'Amy Ryan')]
[('o', 'http://www.wikidata.org/entity/Q238877'), ('oName', 'Jenna Fischer')]
[('o', 'http://www.wikidata.org/entity/Q254766'), ('oName', 'Catherine Tate')]
[('o', 'http://www.wikidata.org/entity/Q2669971'), ('oName', 'Angela Kinsey')]
[('o', 'http://www.wikidata.org/entity/Q2671438'), ('oName', 'Paul Lieberstein')]
[('o', 'http://www.wikidata.org/entity/Q269901'), ('oName', 'Melora Hardin')]
[('o', 'http://www.wikidata.org/entity/Q2924850'), ('oName', 'Brian Baumgartner')]
[('o', 'http://www.wikidata.org/entity/Q296928'), ('oName', 'James Spader')]
[('o', 'http://www.wikidata.org/entity/Q302820

#### Finding relation between cast member and the episodes

In [38]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)')
# wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q216221 ?p ?o.
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?o sc:name ?oName.
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q138996'), ('oName', 'Golden Globe Award for Best Actor – Television Series Musical or Comedy')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q1011547'), ('oName', 'Golden Globe Award')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oName', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oName', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q13235160'), ('oName', 'manufacturer')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q17985761'), ('oName', 'st

In [39]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)')
# wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# -- BACKWARD

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q216221.
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?s sc:name ?sName.
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2529789'), ('sName', 'Screen Actors Guild Award for Outstanding Performance by an Ensemble in a Comedy Series'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q301083'), ('sName', 'Homegrown'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q29051576'), ('sName', 'Beautiful Boy'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q2346771'), ('sName', 'Michael Scott'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q23831'), ('sName', 'The Office'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q7669610'), ('sName', 'TCA Award for Individual Achievement in Comedy'), ('p', 'http://www.wikidata.org/prop/direc

In [40]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)')
# wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# INCREASING LIMIT + FILTERING wdt:P106 ('occupation'), wdt:P166 ('award received'),

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q216221 ?p ?o.
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?o sc:name ?oName.
   }
   FILTER (?p not in (wdt:P106,wdt:P166))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion'), ('o', 'http://www.wikidata.org/entity/Q1841'), ('oName', 'Catholicism')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pName', 'given name'), ('o', 'http://www.wikidata.org/entity/Q4925477'), ('oName', 'John')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('pName', 'residence'), ('o', 'http://www.wikidata.org/entity/Q65'), ('oName', 'Los Angeles')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex 

In [41]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING MORE season --> episodes
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q3465812 ?p ?o. #'The Office, season 1'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?o sc:name ?oName.
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P2747'), ('pName', 'Filmiroda rating'), ('o', 'http://www.wikidata.org/entity/Q23830577'), ('oName', 'Category II')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('oName', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3464665'), ('oName', 'television series season')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q4867642'), ('oName', 'Basketball')]
[('p', 'http://ww

In [44]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
# wd:Q3465812 ('The Office, season 1')
#       --wdt:P527('has part')-->wd:Q4867642('Basketball')
#       --wdt:P527('has part')-->wd:Q5690466('Health Care')
#       --wdt:P527('has part')-->wd:Q7713172('The Alliance')
#       --wdt:P527('has part')-->wd:Q5910010('Hot Girl')
#       --wdt:P527('has part')-->wd:Q5283960('Diversity Day')
#       --wdt:P527('has part')-->wd:Q2199554('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING MORE season --> episodes
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q4867642 ?p ?o. #'Basketball'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?o sc:name ?oName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q21191270'), ('oName', 'television series episode')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pName', 'distribution format'), ('o', 'http://www.wikidata.org/entity/Q723685'), ('oName', 'video on demand')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director'), ('o', 'http://www.wikidata.org/entity/Q1544904'), ('oName', 'Greg Daniels')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('oName', 'The Office')]
[('p', 'ht

In [45]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
# wd:Q3465812 ('The Office, season 1')
#       --wdt:P527('has part')-->wd:Q4867642('Basketball')
#       --wdt:P527('has part')-->wd:Q5690466('Health Care')
#       --wdt:P527('has part')-->wd:Q7713172('The Alliance')
#       --wdt:P527('has part')-->wd:Q5910010('Hot Girl')
#       --wdt:P527('has part')-->wd:Q5283960('Diversity Day')
#       --wdt:P527('has part')-->wd:Q2199554('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING MORE season --> episodes - BACKWARD
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q4867642. #'Basketball'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?s sc:name ?sName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3465812'), ('sName', 'The Office, season 1'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('s', 'http://www.wikidata.org/entity/Q7713172'), ('sName', 'The Alliance'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('s', 'http://www.wikidata.org/entity/Q5910010'), ('sName', 'Hot Girl'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
3


In [46]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING MORE season --> episodes - backward from season
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q3465812. #'The Office, season 1'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?s sc:name ?sName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q23831'), ('sName', 'The Office'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('s', 'http://www.wikidata.org/entity/Q4867642'), ('sName', 'Basketball'), ('p', 'http://www.wikidata.org/prop/direct/P4908'), ('pName', 'season')]
[('s', 'http://www.wikidata.org/entity/Q5690466'), ('sName', 'Health Care'), ('p', 'http://www.wikidata.org/prop/direct/P4908'), ('pName', 'season')]
[('s', 'http://www.wikidata.org/entity/Q7713172'), ('sName', 'The Alliance'), ('p', 'http://www.wikidata.org/prop/direct/P4908'), ('pName', 'season')]
[('s', 'http://www.wikidata.org/entity/Q5910010'), ('sName', 'Hot Girl'), ('p', 'http://www.wikidata.org/prop/direct/P4908'), ('pName', 'season')]
[('s', 'http://www.wikidata.org/entity/Q5283960'), ('sName', 'Diversity Day'), ('p', 'http://www.wikidata.org/prop/direct/P4908'), ('pName', 'season')]
[('s', 'http://www.wikidata.org/entity/Q2199554'), ('sName', 'Pilot'), ('p', 'http://www.wikida

In [47]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING a character - backward
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q1967229. #'Ryan Howard'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?s sc:name ?sName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q23831'), ('sName', 'The Office'), ('p', 'http://www.wikidata.org/prop/direct/P674'), ('pName', 'characters')]
1


In [48]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING a character - forward
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q1967229 ?p ?o. #'Ryan Howard'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?o sc:name ?oName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15632617'), ('oName', 'fictional human')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581097'), ('oName', 'male')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('pName', 'present in work'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('oName', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pName', 'given name'), ('o', 'http://www.wikidata.org/entity/Q3943046'), ('oName', 'Ryan')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer'), ('o', 'http://www.wikidata.org/entity/Q459925'), ('oName', 'B. J. Novak')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pName', 'spouse'), ('o', 'http://www.wikidata.org/entity/Q6386111'), ('oName', 'Kelly Kapoor')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image'), ('o', 'http:/

In [49]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING a character - forward
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q459925 ?p ?o. #'B. J. Novak'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?o sc:name ?oName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q49196'), ('oName', 'Newton')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pName', 'family name'), ('o', 'http://www.wikidata.org/entity/Q15073898'), ('oName', 'Novak')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q7020364'), ('oName', 'Newton South High School')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oName', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oName', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q13371'), ('oName', 'Harvard University')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pNam

In [50]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING a character - backward
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q459925. #'B. J. Novak'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL{
       ?s sc:name ?sName.
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2529789'), ('sName', 'Screen Actors Guild Award for Outstanding Performance by an Ensemble in a Comedy Series'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q2837209'), ('sName', 'Alistair Smythe'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q1967229'), ('sName', 'Ryan Howard'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q23831'), ('sName', 'The Office'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q4254026'), ('sName', 'The Amazing Spider-Man 2'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q20031748'), ('sName', 'The Founder'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', '

In [2]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING a The Office, season 1

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q3465812 ?p ?o. #'The Office, season 1'
   
   # get the label
   ?p sc:name ?pName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), ('pName', 'ČSFD film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2638'), ('pName', 'TV.com ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2704'), ('pName', 'EIDR content ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2747'), ('pName', 'Filmiroda rating')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3302'), ('pName', 'Open Media Database film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('p

In [3]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING a The Office, season 1

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q3465812 ?p ?o. #'The Office, season 1'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P2747'), ('pName', 'Filmiroda rating'), ('o', 'http://www.wikidata.org/entity/Q23830577'), ('oName', 'Category II')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('oName', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3464665'), ('oName', 'television series season')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q4867642'), ('oName', 'Basketball')]
[('p', 'http://ww

#### Trying with HIMYM

In [4]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING HIMYM

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q147235 ?p ?o. #'How I met your mother'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q465556'), ('oName', 'Annie Ilonzeh')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q269891'), ('oName', 'Julianna Guill')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q4888924'), ('oName', 'Benjamin Koldyke')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q446031'), ('oName', 'Nikki Griffin')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q312705'), ('oName', 'John Cho')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q469579'), ('oName', 'Mircea Monroe')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 

In [6]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING HIMYM

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q147235 ?p ?o. #'How I met your mother'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('p', 'http://www.wikida

In [8]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING HIMYM

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?o. #'How I met your mother' --wdt:P527 ('has part')-->
   
   # get the label
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2438066'), ('oName', 'How I Met Your Mother, season 6')]
[('o', 'http://www.wikidata.org/entity/Q2715578'), ('oName', 'How I Met Your Mother, season 1')]
[('o', 'http://www.wikidata.org/entity/Q13567027'), ('oName', 'How I Met Your Mother, season 9')]
[('o', 'http://www.wikidata.org/entity/Q2567330'), ('oName', 'How I Met Your Mother, season 4')]
[('o', 'http://www.wikidata.org/entity/Q338715'), ('oName', 'How I Met Your Mother, season 8')]
[('o', 'http://www.wikidata.org/entity/Q582332'), ('oName', 'How I Met Your Mother, season 5')]
[('o', 'http://www.wikidata.org/entity/Q3468515'), ('oName', 'How I Met Your Mother, season 2')]
[('o', 'http://www.wikidata.org/entity/Q2555117'), ('oName', 'How I Met Your Mother, season 3')]
[('o', 'http://www.wikidata.org/entity/Q2472427'), ('oName', 'How I Met Your Mother, season 7')]
9


In [9]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING HIMYM

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q2715578 ?p ?o. #'How I met your mother, season 1'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q147235'), ('oName', 'How I Met Your Mother')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pName', 'distribution format'), ('o', 'http://www.wikidata.org/entity/Q723685'), ('oName', 'video on demand')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3464665'), ('oName', 'television series season')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pName', 'distribution format'), ('o', 'http://www.wikidata.org/entity/Q54820071'), ('oName', 'digital download')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('pName', 'distributed by'), ('o', 'http://www.wikidata.org/entity/Q9589'), ('oName', 'iTunes')]
[(

In [10]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING HIMYM

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q467447 ?p ?o. #'Pilot'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q200566'), ('oName', 'Cobie Smulders')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q147235'), ('oName', 'How I Met Your Mother')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director'), ('o', 'http://www.wikidata.org/entity/Q460221'), ('oName', 'Pamela Fryman')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q21191270'), ('oName', 'television series episode')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q485310'), ('oName', 'Neil Patrick Harris')]
[('p', 'http:/

#### HIMYM has cast member in the episodes but The Office doesn't

In [11]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
# wd:Q3465812 ('The Office, season 1')
#       --wdt:P527('has part')-->wd:Q4867642('Basketball')
#       --wdt:P527('has part')-->wd:Q5690466('Health Care')
#       --wdt:P527('has part')-->wd:Q7713172('The Alliance')
#       --wdt:P527('has part')-->wd:Q5910010('Hot Girl')
#       --wdt:P527('has part')-->wd:Q5283960('Diversity Day')
#       --wdt:P527('has part')-->wd:Q2199554('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# EXPLORING another episode of The Office (US)

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q5690466 ?p ?o. #'Health Care'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q21191270'), ('oName', 'television series episode')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pName', 'distribution format'), ('o', 'http://www.wikidata.org/entity/Q723685'), ('oName', 'video on demand')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('oName', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter'), ('o', 'http://www.wikidata.org/entity/Q2671438'), ('oName', 'Paul Lieberstein')]
[(

#### HIMYM has cast member in the episodes but The Office doesn't. CONFIRMED.

In [13]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# GETTING ALL CAST MEMBER from the tv series "HIMYM"

queryString = """
SELECT DISTINCT ?castMember ?castMemberName ?episode ?episodeName
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season. #('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?season wdt:P527 ?episode. # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?episode wdt:P161 ?castMember; # wd:Q467447 ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('episode', 'http://www.wikidata.org/entity/Q5264968'), ('episodeName', 'Desperation Day')]
[('castMember', 'http://www.wikidata.org/entity/Q591238'), ('castMemberName', 'Bill Fagerbakke'), ('episode', 'http://www.wikidata.org/entity/Q5264968'), ('episodeName', 'Desperation Day')]
[('castMember', 'http://www.wikidata.org/entity/Q1189102'), ('castMemberName', 'Nazanin Boniadi'), ('episode', 'http://www.wikidata.org/entity/Q5264968'), ('episodeName', 'Desperation Day')]
[('castMember', 'http://www.wikidata.org/entity/Q199929'), ('castMemberName', 'Jennifer Morrison'), ('episode', 'http://www.wikidata.org/entity/Q5264968'), ('episodeName', 'Desperation Day')]
[('castMember', 'http://www.wikidata.org/entity/Q978367'), ('castMemberName', 'Jesse Heiman'), ('episode', 'http://www.wikidata.org/entity/Q5264968'), ('episodeName', 'Desperation Day')]
[('castMember', 'http://www.wikidata.org/en

In [14]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# GETTING ALL CAST MEMBER from the tv series "HIMYM"
# ORDERING

queryString = """
SELECT DISTINCT ?castMember ?castMemberName ?episode ?episodeName
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season. #('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?season wdt:P527 ?episode. # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?episode wdt:P161 ?castMember; # wd:Q467447 ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
ORDER BY ASC(?castMemberName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('episode', 'http://www.wikidata.org/entity/Q16745474'), ('episodeName', 'Noretta')]
[('castMember', 'http://www.wikidata.org/entity/Q667979'), ('castMemberName', 'Aaron Hill'), ('episode', 'http://www.wikidata.org/entity/Q7084332'), ('episodeName', 'Old King Clancy')]
[('castMember', 'http://www.wikidata.org/entity/Q321131'), ('castMemberName', 'Abigail Spencer'), ('episode', 'http://www.wikidata.org/entity/Q16744365'), ('episodeName', 'How I Met Everyone Else')]
[('castMember', 'http://www.wikidata.org/entity/Q151866'), ('castMemberName', 'Adriana Lima'), ('episode', 'http://www.wikidata.org/entity/Q7776702'), ('episodeName', 'The Yips')]
[('castMember', 'http://www.wikidata.org/entity/Q2827976'), ('castMemberName', 'Ailsa Marshall'), ('episode', 'http://www.wikidata.org/entity/Q6927048'), ('episodeName', 'Moving Day')]
[('castMember', 'http://www.wikidata.org/entity/Q2636696')

In [15]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# GETTING ALL CAST MEMBER from the tv series "HIMYM"
# ORDERING
# GROUPING

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?episode) AS ?episodes)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season. #('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?season wdt:P527 ?episode. # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?episode wdt:P161 ?castMember; # wd:Q467447 ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?episodes)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q202304'), ('castMemberName', 'Jason Segel'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q199927'), ('castMemberName', 'Alyson Hannigan'), ('episodes', '143')]
[('castMember', 'http://www.wikidata.org/entity/Q333544'), ('castMemberName', 'Bob Saget'), ('episodes', '142')]
[('castMember', 'http://www.wikidata.org/entity/Q229914'), ('castMemberName', 'Lyndsy Fonseca'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q16149506

In [16]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# GETTING ALL CAST MEMBER from the tv series "HIMYM"
# ORDERING
# GROUPING - checking if count(*) is the same as count(?episode)

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(*) AS ?episodes)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season. #('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?season wdt:P527 ?episode. # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?episode wdt:P161 ?castMember; # wd:Q467447 ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?episodes)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q202304'), ('castMemberName', 'Jason Segel'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q199927'), ('castMemberName', 'Alyson Hannigan'), ('episodes', '143')]
[('castMember', 'http://www.wikidata.org/entity/Q333544'), ('castMemberName', 'Bob Saget'), ('episodes', '142')]
[('castMember', 'http://www.wikidata.org/entity/Q229914'), ('castMemberName', 'Lyndsy Fonseca'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q16149506

In [17]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# GETTING ALL CAST MEMBER from the tv series "HIMYM"
# ORDERING
# GROUPING
# TRYING THE SAME WITH The Office (US)

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?episode) AS ?episodes)
WHERE {
   # bind something
   wd:Q23831 wdt:P527 ?season. 
   ?season wdt:P527 ?episode. 
   ?episode wdt:P161 ?castMember;
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?episodes)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q926912'), ('castMemberName', 'Craig Robinson'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q349548'), ('castMemberName', 'Rainn Wilson'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q238877'), ('castMemberName', 'Jenna Fischer'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q313039'), ('castMemberName', 'John Krasinski'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q328790'), ('castMemberName', 'Ed Helms'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q254766'), ('castMemberName', 'Catherine Tate'), ('episodes', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q72077'), ('castMemberName', 'Ellie Kemper'), ('episodes', '1')]
7


In [18]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
#                               --wdt:P161 ('cast member')--> wd:Q216221 ('Steve Carell')
#                               --wdt:P674 ('characters')--> wd:Q1967229 ('Ryan Howard')
# wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')

# finding the cast members 
# finding relation between cast member and the episode of 'The office (US)'
# GETTING ALL CAST MEMBER from the tv series "HIMYM"
# ORDERING
# GROUPING
# TRYING THE SAME WITH The Office (US)

queryString = """
SELECT DISTINCT ?castMember ?castMemberName ?episode ?episodeName
WHERE {
   # bind something
   wd:Q23831 wdt:P527 ?season. 
   ?season wdt:P527 ?episode. 
   ?episode wdt:P161 ?castMember;
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
ORDER BY DESC(?episodes)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q328790'), ('castMemberName', 'Ed Helms'), ('episode', 'http://www.wikidata.org/entity/Q4760324'), ('episodeName', "Andy's Ancestry")]
[('castMember', 'http://www.wikidata.org/entity/Q254766'), ('castMemberName', 'Catherine Tate'), ('episode', 'http://www.wikidata.org/entity/Q4760324'), ('episodeName', "Andy's Ancestry")]
[('castMember', 'http://www.wikidata.org/entity/Q72077'), ('castMemberName', 'Ellie Kemper'), ('episode', 'http://www.wikidata.org/entity/Q4760324'), ('episodeName', "Andy's Ancestry")]
[('castMember', 'http://www.wikidata.org/entity/Q238877'), ('castMemberName', 'Jenna Fischer'), ('episode', 'http://www.wikidata.org/entity/Q4760324'), ('episodeName', "Andy's Ancestry")]
[('castMember', 'http://www.wikidata.org/entity/Q349548'), ('castMemberName', 'Rainn Wilson'), ('episode', 'http://www.wikidata.org/entity/Q4760324'), ('episodeName', "Andy's Ancestry")]
[('castMember', 'http://www.wikidata.org/entity/Q926912'), 

In [20]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')

# REMOVED LIMIT

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?episode) AS ?episodes)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode.     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?episodes)
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q202304'), ('castMemberName', 'Jason Segel'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q199927'), ('castMemberName', 'Alyson Hannigan'), ('episodes', '143')]
[('castMember', 'http://www.wikidata.org/entity/Q333544'), ('castMemberName', 'Bob Saget'), ('episodes', '142')]
[('castMember', 'http://www.wikidata.org/entity/Q229914'), ('castMemberName', 'Lyndsy Fonseca'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q16149506

### Final query for this task

#### Who are the most present actors? ASSUMING atleast 4 appearances (to limit the result)

In [22]:
# Get the number of episodes in which the cast members played a role. Who are the most present actors?
#  (the result set must be a list of triples actor/actress IRI, label and #episodes)

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?episode) AS ?episodes)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode.     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
}
GROUP BY ?castMember ?castMemberName
HAVING (COUNT(?episode) > 3) #Who are the most present actors?
ORDER BY DESC(?episodes)
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q202304'), ('castMemberName', 'Jason Segel'), ('episodes', '145')]
[('castMember', 'http://www.wikidata.org/entity/Q199927'), ('castMemberName', 'Alyson Hannigan'), ('episodes', '143')]
[('castMember', 'http://www.wikidata.org/entity/Q333544'), ('castMemberName', 'Bob Saget'), ('episodes', '142')]
[('castMember', 'http://www.wikidata.org/entity/Q229914'), ('castMemberName', 'Lyndsy Fonseca'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q297128'), ('castMemberName', 'David Henrie'), ('episodes', '48')]
[('castMember', 'http://www.wikidata.org/entity/Q16149506

## Task 4
Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

### Possible Solution:
1) Getting film's date done by a Cast member of the HIMYM tv series
2) Checking if the films date overlap with any HIMYM season
3) Extend to all the cast member of HIMYM and return the only actor who acted in more films

#### Part 1

##### Finding Films from a cast member of HIMYM

In [23]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q223455 ?p ?o. #'Josh Radnor'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q1797768'), ('oName', 'Kenyon College')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oName', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oName', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q16567'), ('oName', 'Columbus')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q2259451'), ('oName', 'stage actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q2405480'), ('oName', 'voice actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occu

In [24]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q223455. #'Josh Radnor'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?s sc:name ?sName
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q147235'), ('sName', 'How I Met Your Mother'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q5264968'), ('sName', 'Desperation Day'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q5521981'), ('sName', 'Garbage Island'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q7080458'), ('sName', 'Oh Honey'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q5888848'), ('sName', 'Home Wreckers'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q7858076'), ('sName', 'Twin Beds'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q8074081'), ('sN

In [25]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD
# FILTERing cast member to check if there is anything else

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q223455. #'Josh Radnor'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?s sc:name ?sName
   }
   
   FILTER (?p not in (wdt:P161))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q50383276'), ('sName', 'Mexican Wrestler Ted'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q775254'), ('sName', 'Ted Mosby'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q50378099'), ('sName', 'Future Ted'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q4137975'), ('sName', 'Liberal Arts'), ('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director')]
[('s', 'http://www.wikidata.org/entity/Q1050720'), ('sName', 'Happythankyoumoreplease'), ('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director')]
[('s', 'http://www.wikidata.org/entity/Q4137975'), ('sName', 'Liberal Arts'), ('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter')]
[('s', 'http://www.wikidata.org/entity/Q1050720'), ('sName', 

In [25]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD
# FILTERing cast member to check if there is anything else

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q223455. #'Josh Radnor'
   
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?s sc:name ?sName
   }
   
   FILTER (?p not in (wdt:P161))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q50383276'), ('sName', 'Mexican Wrestler Ted'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q775254'), ('sName', 'Ted Mosby'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q50378099'), ('sName', 'Future Ted'), ('p', 'http://www.wikidata.org/prop/direct/P175'), ('pName', 'performer')]
[('s', 'http://www.wikidata.org/entity/Q4137975'), ('sName', 'Liberal Arts'), ('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director')]
[('s', 'http://www.wikidata.org/entity/Q1050720'), ('sName', 'Happythankyoumoreplease'), ('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director')]
[('s', 'http://www.wikidata.org/entity/Q4137975'), ('sName', 'Liberal Arts'), ('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter')]
[('s', 'http://www.wikidata.org/entity/Q1050720'), ('sName', 

In [26]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       sc:name ?filmName;
       ?p ?o.
       
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop

In [27]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       sc:name ?filmName;
       ?p ?o.
       
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop

In [28]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       sc:name ?filmName;
       ?p ?o.
       
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 80
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop

In [29]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       sc:name ?filmName;
       wdt:P31 ?o. # 'instance of'
       
   # get the label
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 80
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11424'), ('oName', 'film')]
[('o', 'http://www.wikidata.org/entity/Q21191270'), ('oName', 'television series episode')]
[('o', 'http://www.wikidata.org/entity/Q5398426'), ('oName', 'television series')]
3


In [31]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs

queryString = """
SELECT DISTINCT ?instance ?instanceName (COUNT(*) as ?tot)
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       sc:name ?filmName;
       wdt:P31 ?instance. # 'instance of'
       
   # get the label
   OPTIONAL {
       ?instance sc:name ?instanceName
   }
}
GROUP BY ?instance ?instanceName
LIMIT 80
"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q11424'), ('instanceName', 'film'), ('tot', '7')]
[('instance', 'http://www.wikidata.org/entity/Q21191270'), ('instanceName', 'television series episode'), ('tot', '145')]
[('instance', 'http://www.wikidata.org/entity/Q5398426'), ('instanceName', 'television series'), ('tot', '3')]
3


In [32]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs

queryString = """
SELECT DISTINCT ?film ?filmName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName.
       
   # get the label
   OPTIONAL {
       #?instance sc:name ?instanceName
   }
}
LIMIT 80
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q4137975'), ('filmName', 'Liberal Arts')]
[('film', 'http://www.wikidata.org/entity/Q4690774'), ('filmName', 'Afternoon Delight')]
[('film', 'http://www.wikidata.org/entity/Q52715605'), ('filmName', 'Social Animals')]
[('film', 'http://www.wikidata.org/entity/Q18170028'), ('filmName', 'The Galapagos Affair')]
[('film', 'http://www.wikidata.org/entity/Q1050720'), ('filmName', 'Happythankyoumoreplease')]
[('film', 'http://www.wikidata.org/entity/Q1145848'), ('filmName', 'Not Another Teen Movie')]
[('film', 'http://www.wikidata.org/entity/Q2884686'), ('filmName', "Barney's Great Adventure")]
7


In [33]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs
# GETTING to DATE for cheching "while condition"

queryString = """
SELECT DISTINCT ?film ?filmName ?p ?pName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       ?p ?o.
       
   # get the label
   ?p sc:name ?pName
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 80
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q4137975'), ('filmName', 'Liberal Arts'), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor')]
[('film', 'http://www.wikidata.org/entity/Q1050720'), ('filmName', 'Happythankyoumoreplease'), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor')]
[('film', 'http://www.wikidata.org/entity/Q1145848'), ('filmName', 'Not Another Teen Movie'), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor')]
[('film', 'http://www.wikidata.org/entity/Q2884686'), ('filmName', "Barney's Great Adventure"), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor')]
[('film', 'http://www.wikidata.org/entity/Q4137975'), ('filmName', 'Liberal Arts'), ('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pName', 'Box Office Mojo film ID (former scheme)')]
[('film', 'http://www.wikidata.org/entity/Q4690774'), ('filmName', 'Afternoon Delight'), ('p', 'http://www.wikidata.org/p

##### Finding Film's date

In [34]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs
# GETTING to DATE for cheching "while condition"

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       ?p ?o.
       
   # get the label
   ?p sc:name ?pName
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 80
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pName', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pName', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pName', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1804'), ('pName', 'DNF film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('pName', 'Netflix ID')]
[('p', 'http://

In [35]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs
# GETTING to DATE for cheching "while condition"

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       ?p ?o.
       
   # get the label
   ?p sc:name ?pName
   OPTIONAL {
       ?o sc:name ?oName
   }
}
LIMIT 160
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pName', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pName', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pName', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pName', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1804'), ('pName', 'DNF film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('pName', 'Netflix ID')]
[('p', 'http://

In [36]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# finding films starting from one cast member of HIMYM
# BACKWARD
# GETTING TO FILMs
# GETTING to DATE for cheching "while condition"
# GOT publication date

queryString = """
SELECT DISTINCT ?film ?filmName ?date
WHERE {
   # bind something
   ?film wdt:P161 wd:Q223455; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       wdt:P577 ?date. # 'publication date'
}
LIMIT 160
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q4137975'), ('filmName', 'Liberal Arts'), ('date', '2012-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q4690774'), ('filmName', 'Afternoon Delight'), ('date', '2013-01-21T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q4690774'), ('filmName', 'Afternoon Delight'), ('date', '2015-05-07T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q52715605'), ('filmName', 'Social Animals'), ('date', '2018-06-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18170028'), ('filmName', 'The Galapagos Affair'), ('date', '2013-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1050720'), ('filmName', 'Happythankyoumoreplease'), ('date', '2010-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1145848'), ('filmName', 'Not Another Teen Movie'), ('date', '2001-12-07T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1145848'), ('filmName', 'Not Another Teen Movie'), ('date', '2001-12-14T00:00:00

#### Part 2

##### Getting Seasons' dates
Getting HIMYM season' dates to compare with film publication date

In [40]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# FROM seasong trying to get the date to compare with film pubblication date

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       ?p ?o.
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
   # get the label
   ?castMember sc:name ?castMemberName.
   
   ?p sc:name ?pName
}
#GROUP BY ?castMember ?castMemberName
#HAVING (COUNT(?episode) > 3) #Who are the most present actors?
#ORDER BY DESC(?episodes)
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), ('pName', 'ČSFD film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2638'), ('pName', 'TV.com ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2704'), ('pName', 'EIDR content ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3302'), ('pName', 'Open Media Database film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pName'

In [41]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# FROM seasong trying to get the date to compare with film pubblication date

queryString = """
SELECT DISTINCT ?season ?seasonName ?start ?end
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       sc:name ?seasonName;           # wdt:P580 ('start time')
       wdt:P580 ?start;           # wdt:P580 ('start time')
       wdt:P582 ?end.             # wdt:P582 ('end time')
       
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.
           
   # get the label
   ?castMember sc:name ?castMemberName.
}
#GROUP BY ?castMember ?castMemberName
#HAVING (COUNT(?episode) > 3) #Who are the most present actors?
#ORDER BY DESC(?episodes)
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('start', '2010-09-20T00:00:00Z'), ('end', '2011-05-16T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q2715578'), ('seasonName', 'How I Met Your Mother, season 1'), ('start', '2005-09-19T00:00:00Z'), ('end', '2006-05-15T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q13567027'), ('seasonName', 'How I Met Your Mother, season 9'), ('start', '2013-09-23T00:00:00Z'), ('end', '2014-03-31T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q2567330'), ('seasonName', 'How I Met Your Mother, season 4'), ('start', '2008-09-22T00:00:00Z'), ('end', '2009-05-18T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q338715'), ('seasonName', 'How I Met Your Mother, season 8'), ('start', '2012-09-24T00:00:00Z'), ('end', '2013-05-13T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q582332'), ('seasonName', 'How I Met Your Mother, season 5'), ('start', '

##### Comparing
If film publication date is in between season start time and end time

In [42]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging

queryString = """
SELECT DISTINCT ?season ?seasonName ?start ?end ?castMember ?castMemberName ?filmName ?date
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       sc:name ?seasonName;           # wdt:P580 ('start time')
       wdt:P580 ?start;           # wdt:P580 ('start time')
       wdt:P582 ?end.             # wdt:P582 ('end time')
       
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
       sc:name ?episodeName.
           
   ?film wdt:P161 ?castMember; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       wdt:P577 ?date. # 'publication date'
           
   # get the label
   ?castMember sc:name ?castMemberName.
   
   FILTER (?castMember in (wd:Q223455)) #; #'Josh Radnor'
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('start', '2010-09-20T00:00:00Z'), ('end', '2011-05-16T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('filmName', 'Liberal Arts'), ('date', '2012-01-01T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('start', '2010-09-20T00:00:00Z'), ('end', '2011-05-16T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('filmName', 'Afternoon Delight'), ('date', '2013-01-21T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('start', '2010-09-20T00:00:00Z'), ('end', '2011-05-16T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('filmName', 'Afternoon Delight'), ('date', '2015-05-07

In [43]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date

queryString = """
SELECT DISTINCT ?season ?seasonName ?start ?end ?castMember ?castMemberName ?filmName ?date
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       sc:name ?seasonName;           # wdt:P580 ('start time')
       wdt:P580 ?start;           # wdt:P580 ('start time')
       wdt:P582 ?end.             # wdt:P582 ('end time')
       
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
       sc:name ?episodeName.
           
   ?film wdt:P161 ?castMember; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       wdt:P577 ?date. # 'publication date'
           
   # get the label
   ?castMember sc:name ?castMemberName.
   
   FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
   FILTER (?date >= ?start && ?date <= ?end)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q582332'), ('seasonName', 'How I Met Your Mother, season 5'), ('start', '2009-09-21T00:00:00Z'), ('end', '2010-05-24T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('filmName', 'Happythankyoumoreplease'), ('date', '2010-01-01T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q2472427'), ('seasonName', 'How I Met Your Mother, season 7'), ('start', '2011-09-19T00:00:00Z'), ('end', '2012-05-14T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('filmName', 'Liberal Arts'), ('date', '2012-01-01T00:00:00Z')]
2


#### PART 3

##### Extending to all cast members

In [44]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member

queryString = """
SELECT DISTINCT ?season ?seasonName ?start ?end ?castMember ?castMemberName ?filmName ?date
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       sc:name ?seasonName;           # wdt:P580 ('start time')
       wdt:P580 ?start;           # wdt:P580 ('start time')
       wdt:P582 ?end.             # wdt:P582 ('end time')
       
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
       sc:name ?episodeName.
           
   ?film wdt:P161 ?castMember; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       wdt:P577 ?date. # 'publication date'
           
   # get the label
   ?castMember sc:name ?castMemberName.
   
   # FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
   FILTER (?date >= ?start && ?date <= ?end)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2567330'), ('seasonName', 'How I Met Your Mother, season 4'), ('start', '2008-09-22T00:00:00Z'), ('end', '2009-05-18T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('filmName', "The Slammin' Salmon"), ('date', '2009-01-01T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q2555117'), ('seasonName', 'How I Met Your Mother, season 3'), ('start', '2007-09-24T00:00:00Z'), ('end', '2008-05-19T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q312705'), ('castMemberName', 'John Cho'), ('filmName', 'Harold & Kumar Escape from Guantanamo Bay'), ('date', '2008-03-23T00:00:00Z')]
[('season', 'http://www.wikidata.org/entity/Q2555117'), ('seasonName', 'How I Met Your Mother, season 3'), ('start', '2007-09-24T00:00:00Z'), ('end', '2008-05-19T00:00:00Z'), ('castMember', 'http://www.wikidata.org/entity/Q312705'), ('castMemberName', 'John Cho'), ('filmName', "Nick and Nora

In [46]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member
# GROUPING 

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(*) AS ?films)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       sc:name ?seasonName;           # wdt:P580 ('start time')
       wdt:P580 ?start;           # wdt:P580 ('start time')
       wdt:P582 ?end.             # wdt:P582 ('end time')
       
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
       sc:name ?episodeName.
           
   ?film wdt:P161 ?castMember; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       wdt:P577 ?date. # 'publication date'
           
   # get the label
   ?castMember sc:name ?castMemberName.
   
   # FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
   FILTER (?date >= ?start && ?date <= ?end)
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('films', '248')]
[('castMember', 'http://www.wikidata.org/entity/Q202304'), ('castMemberName', 'Jason Segel'), ('films', '211')]
[('castMember', 'http://www.wikidata.org/entity/Q199927'), ('castMemberName', 'Alyson Hannigan'), ('films', '96')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('films', '71')]
[('castMember', 'http://www.wikidata.org/entity/Q229914'), ('castMemberName', 'Lyndsy Fonseca'), ('films', '60')]
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('films', '31')]
[('castMember', 'http://www.wikidata.org/entity/Q591238'), ('castMemberName', 'Bill Fagerbakke'), ('films', '29')]
[('castMember', 'http://www.wikidata.org/entity/Q333544'), ('castMemberName', 'Bob Saget'), ('films', '22')]
[('castMember', 'http://www.wikidata.org/entity/Q888178'), ('castMemberName', 'Bob

##### Problem duplicates: Josh Radnor should have 2 but he has 31

In [47]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member
# GROUPING 
# SOLVING DUPLICATES -> Josh Radnor HAS 31 instead of 2

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(*) AS ?films)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       sc:name ?seasonName;           # wdt:P580 ('start time')
       wdt:P580 ?start;           # wdt:P580 ('start time')
       wdt:P582 ?end.             # wdt:P582 ('end time')
       
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
       sc:name ?episodeName.
           
   ?film wdt:P161 ?castMember; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       wdt:P577 ?date. # 'publication date'
           
   # get the label
   ?castMember sc:name ?castMemberName.
   
   FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
   FILTER (?date >= ?start && ?date <= ?end)
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('films', '31')]
1


In [48]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member
# GROUPING 
# SOLVING DUPLICATES -> Josh Radnor HAS 31 instead of 2

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?season) AS ?films)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
   ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
       sc:name ?seasonName;           # wdt:P580 ('start time')
       wdt:P580 ?start;           # wdt:P580 ('start time')
       wdt:P582 ?end.             # wdt:P582 ('end time')
       
   ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
       sc:name ?episodeName.
           
   ?film wdt:P161 ?castMember; #'Josh Radnor'
       wdt:P31 wd:Q11424; # 'instance of'
       sc:name ?filmName;
       wdt:P577 ?date. # 'publication date'
           
   # get the label
   ?castMember sc:name ?castMemberName.
   
   FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
   FILTER (?date >= ?start && ?date <= ?end)
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('films', '31')]
1


In [49]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member
# GROUPING 
# SOLVING DUPLICATES -> Josh Radnor HAS 31 instead of 2

queryString = """
SELECT DISTINCT *
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?film ?filmName
    WHERE {
       # bind something
       wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
       ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
           sc:name ?seasonName;           # wdt:P580 ('start time')
           wdt:P580 ?start;           # wdt:P580 ('start time')
           wdt:P582 ?end.             # wdt:P582 ('end time')

       ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.

       ?film wdt:P161 ?castMember; #'Josh Radnor'
           wdt:P31 wd:Q11424; # 'instance of'
           sc:name ?filmName;
           wdt:P577 ?date. # 'publication date'

       # get the label
       ?castMember sc:name ?castMemberName.

       FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
       FILTER (?date >= ?start && ?date <= ?end)
    }
    #GROUP BY ?castMember ?castMemberName
}
#ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('film', 'http://www.wikidata.org/entity/Q1050720'), ('filmName', 'Happythankyoumoreplease')]
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('film', 'http://www.wikidata.org/entity/Q4137975'), ('filmName', 'Liberal Arts')]
2


##### SOL outer query it solves the problem

In [50]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member
# GROUPING 
# SOLVING DUPLICATES -> Josh Radnor HAS 31 instead of 2

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(*) AS ?films)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?film ?filmName
    WHERE {
       # bind something
       wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
       ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
           sc:name ?seasonName;           # wdt:P580 ('start time')
           wdt:P580 ?start;           # wdt:P580 ('start time')
           wdt:P582 ?end.             # wdt:P582 ('end time')

       ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.

       ?film wdt:P161 ?castMember; #'Josh Radnor'
           wdt:P31 wd:Q11424; # 'instance of'
           sc:name ?filmName;
           wdt:P577 ?date. # 'publication date'

       # get the label
       ?castMember sc:name ?castMemberName.

       FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
       FILTER (?date >= ?start && ?date <= ?end)
    }
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q223455'), ('castMemberName', 'Josh Radnor'), ('films', '2')]
1


In [51]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member
# GROUPING 
# SOLVING DUPLICATES -> Josh Radnor HAS 31 instead of 2

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(*) AS ?films)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?film ?filmName
    WHERE {
       # bind something
       wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
       ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
           sc:name ?seasonName;           # wdt:P580 ('start time')
           wdt:P580 ?start;           # wdt:P580 ('start time')
           wdt:P582 ?end.             # wdt:P582 ('end time')

       ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.

       ?film wdt:P161 ?castMember; #'Josh Radnor'
           wdt:P31 wd:Q11424; # 'instance of'
           sc:name ?filmName;
           wdt:P577 ?date. # 'publication date'

       # get the label
       ?castMember sc:name ?castMemberName.

       #FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
       FILTER (?date >= ?start && ?date <= ?end)
    }
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('films', '10')]
[('castMember', 'http://www.wikidata.org/entity/Q202304'), ('castMemberName', 'Jason Segel'), ('films', '9')]
[('castMember', 'http://www.wikidata.org/entity/Q192165'), ('castMemberName', 'Danny Glover'), ('films', '6')]
[('castMember', 'http://www.wikidata.org/entity/Q591238'), ('castMemberName', 'Bill Fagerbakke'), ('films', '6')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('films', '6')]
[('castMember', 'http://www.wikidata.org/entity/Q229669'), ('castMemberName', 'Malin Åkerman'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q236189'), ('castMemberName', 'Judy Greer'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q1280821'), ('castMemberName', 'Richard Gant'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q229914'), ('castMemberName', 'Lyndsy Fonsec

In [52]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wd:Q223455 ('Josh Radnor'), ('episodes', '145')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

# CONNECTING THE TWO PARTS, FIXING one cast member for debugging
# ADDED FILTER to date
# Removing fixed cast member
# GROUPING 
# SOLVING DUPLICATES -> Josh Radnor HAS 31 instead of 2

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?film) AS ?films)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?film ?filmName
    WHERE {
       # bind something
       wd:Q147235 wdt:P527 ?season.   # wd:Q147235  ('How I met your mother') --wdt:P527 ('has part')--> wd:Q2715578 ('How I Met Your Mother, season 1')
       ?season wdt:P527 ?episode;     # wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')--> wd:Q467447 ('Pilot')
           sc:name ?seasonName;           # wdt:P580 ('start time')
           wdt:P580 ?start;           # wdt:P580 ('start time')
           wdt:P582 ?end.             # wdt:P582 ('end time')

       ?episode wdt:P161 ?castMember; # wd:Q467447  ('Pilot') --wdt:P161('cast member')-->  wd:Q485310 ('Neil Patrick Harris')
           sc:name ?episodeName.

       ?film wdt:P161 ?castMember; #'Josh Radnor'
           wdt:P31 wd:Q11424; # 'instance of'
           sc:name ?filmName;
           wdt:P577 ?date. # 'publication date'

       # get the label
       ?castMember sc:name ?castMemberName.

       #FILTER (?castMember in (wd:Q223455)) #'Josh Radnor'
       FILTER (?date >= ?start && ?date <= ?end)
    }
}
GROUP BY ?castMember ?castMemberName
HAVING (COUNT(?film)>=2) #we want more films  --> top N --> atleast 2 films
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('films', '10')]
[('castMember', 'http://www.wikidata.org/entity/Q202304'), ('castMemberName', 'Jason Segel'), ('films', '9')]
[('castMember', 'http://www.wikidata.org/entity/Q192165'), ('castMemberName', 'Danny Glover'), ('films', '6')]
[('castMember', 'http://www.wikidata.org/entity/Q591238'), ('castMemberName', 'Bill Fagerbakke'), ('films', '6')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders'), ('films', '6')]
[('castMember', 'http://www.wikidata.org/entity/Q229669'), ('castMemberName', 'Malin Åkerman'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q236189'), ('castMemberName', 'Judy Greer'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q1280821'), ('castMemberName', 'Richard Gant'), ('films', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q229914'), ('castMemberName', 'Lyndsy Fonsec

##### Return only one actor

In [55]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?film) AS ?films)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?film
    WHERE {
       # bind something
       wd:Q147235 wdt:P527 ?season.       # wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')-->
       ?season wdt:P527 ?episode;         # --wdt:P527 ('has part')----> 
           sc:name ?seasonName;
           wdt:P580 ?startSeason;         # --wdt:P580 ('start time')-->
           wdt:P582 ?endSeason.           # --wdt:P582 ('end time')---->

       ?episode wdt:P161 ?castMember;     # --wdt:P161('cast member')-->
           sc:name ?episodeName.

       ?castMember sc:name ?castMemberName.

       ?film wdt:P161 ?castMember;
           wdt:P31 wd:Q11424;             # --wdt:P31 ('instance of')--> wd:Q11424 ('film')
           sc:name ?filmName;
           wdt:P577 ?filmPublicationDate. # --wdt:P577 ('publication date')-->

       FILTER (?filmPublicationDate >= ?startSeason && ?filmPublicationDate <= ?endSeason)
    }
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('films', '10')]
1


### Final query for this task

In [55]:
# Check who is the actor who acted in more films while working on "How I met your mother"
#  (the result set must be a list of triples actor/actress IRI, label and #films).

# wd:Q147235 ('How I met your mother')            --wdt:P527 ('has part')----> wd:Q2715578 ('How I Met Your Mother, season 1')
# wd:Q2715578 ('How I Met Your Mother, season 1') --wdt:P527 ('has part')----> wd:Q467447 ('Pilot')
# wd:Q467447 ('Pilot')                            --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
# wdt:P31 ('instance of')--> wd:Q11424 ('film')
# wdt:P577 ('publication date')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(?film) AS ?films)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?film
    WHERE {
       # bind something
       wd:Q147235 wdt:P527 ?season.       # wd:Q147235 ('How I met your mother') --wdt:P527 ('has part')-->
       ?season wdt:P527 ?episode;         # --wdt:P527 ('has part')----> 
           sc:name ?seasonName;
           wdt:P580 ?startSeason;         # --wdt:P580 ('start time')-->
           wdt:P582 ?endSeason.           # --wdt:P582 ('end time')---->

       ?episode wdt:P161 ?castMember;     # --wdt:P161('cast member')-->
           sc:name ?episodeName.

       ?castMember sc:name ?castMemberName.

       ?film wdt:P161 ?castMember;
           wdt:P31 wd:Q11424;             # --wdt:P31 ('instance of')--> wd:Q11424 ('film')
           sc:name ?filmName;
           wdt:P577 ?filmPublicationDate. # --wdt:P577 ('publication date')-->

       FILTER (?filmPublicationDate >= ?startSeason && ?filmPublicationDate <= ?endSeason)
    }
}
GROUP BY ?castMember ?castMemberName
ORDER BY DESC(?films)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q485310'), ('castMemberName', 'Neil Patrick Harris'), ('films', '10')]
1


## Task 5
Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

In [56]:
# Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members 
#    (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')

# From task 2

queryString = """
SELECT DISTINCT ?season ?seasonName ?episodes
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P527 ?season.  #--wdt:P527 ('has part')-->
   ?season wdt:P1113 ?episodes. #--wdt:P1113 ('number of episodes')-->
   
   # get the label
   ?season sc:name ?seasonName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2715578'), ('seasonName', 'How I Met Your Mother, season 1'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q3468515'), ('seasonName', 'How I Met Your Mother, season 2'), ('episodes', '22')]
[('season', 'http://www.wikidata.org/entity/Q2555117'), ('seasonName', 'How I Met Your Mother, season 3'), ('episodes', '20')]
[('season', 'http://www.wikidata.org/entity/Q2567330'), ('seasonName', 'How I Met Your Mother, season 4'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q582332'), ('seasonName', 'How I Met Your Mother, season 5'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2438066'), ('seasonName', 'How I Met Your Mother, season 6'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q2472427'), ('seasonName', 'How I Met Your Mother, season 7'), ('episodes', '24')]
[('season', 'http://www.wikidata.org/entity/Q338715'), ('seasonName', 'How I Met Your Mother, season 8')

In [57]:
# Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members 
#    (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')

# From task 2

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P527 ?season;  #--wdt:P527 ('has part')-->
           ?p ?o.
   
   ?season wdt:P1113 ?episodes. #--wdt:P1113 ('number of episodes')-->
   
   # get the label
   ?season sc:name ?seasonName.
   ?p sc:name ?pName.
}
ORDER BY ASC(?seasonName)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('p', 'http://www.wikida

In [58]:
# Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members 
#    (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')

# From task 2
# FINDING if there is a dataproperty for seasons, episods and cast member direclty from the tv series

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P527 ?season;  #--wdt:P527 ('has part')-->
           ?p ?o.
   
   ?season wdt:P1113 ?episodes. #--wdt:P1113 ('number of episodes')-->
   
   # get the label
   ?season sc:name ?seasonName.
   ?p sc:name ?pName.
}
ORDER BY ASC(?seasonName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('p', 'http://www.wikida

In [59]:
# Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members 
#    (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')

# wdt:P2437 ('number of seasons')
# wdt:P1113 ('number of episodes')
# wdt:P161 ('cast member')

# From task 2
# FINDING if there is a dataproperty for seasons, episods and cast member direclty from the tv series

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?seasons ?episodes ?castMember
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P2437 ?seasons;  #--wdt:P527 ('number of seasons')---->
       wdt:P1113 ?episodes;       #--wdt:P1113 ('number of episodes')-->
       wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
       sc:name ?tvSeriesName.
}
ORDER BY ASC(?seasonName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('seasons', '9'), ('episodes', '208'), ('castMember', 'http://www.wikidata.org/entity/Q465556')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('seasons', '9'), ('episodes', '208'), ('castMember', 'http://www.wikidata.org/entity/Q269891')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('seasons', '9'), ('episodes', '208'), ('castMember', 'http://www.wikidata.org/entity/Q4888924')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('seasons', '9'), ('episodes', '208'), ('castMember', 'http://www.wikidata.org/entity/Q446031')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('seasons', '9'), ('episodes', '208'), ('castMember', 'http://www.wikidata.org/entity/Q312705')]
[('tvS

In [60]:
# Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members 
#    (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')

# wdt:P2437 ('number of seasons')
# wdt:P1113 ('number of episodes')
# wdt:P161 ('cast member')

# From task 2
# FINDING if there is a dataproperty for seasons, episods and cast member direclty from the tv series
# GROUPING

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?seasons ?episodes (COUNT (?castMember) AS ?castMembers)
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P2437 ?seasons;  #--wdt:P527 ('number of seasons')---->
       wdt:P1113 ?episodes;       #--wdt:P1113 ('number of episodes')-->
       wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
       sc:name ?tvSeriesName.
}
GROUP BY ?tvSeries ?tvSeriesName ?seasons ?episodes
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('seasons', '9'), ('episodes', '201'), ('castMembers', '25')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('seasons', '9'), ('episodes', '208'), ('castMembers', '480')]
2


In [61]:
# Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members 
#    (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

# wd:Q23831 ('The Office (US)') --wdt:P527 ('has part')--> wd:Q3465812 ('The Office, season 1')
# wd:Q3465812 ('The Office, season 1') --wdt:P1113 ('number of episodes')-> ('o', '6')

# wdt:P2437 ('number of seasons')
# wdt:P1113 ('number of episodes')
# wdt:P161 ('cast member')

# From task 2
# FINDING if there is a dataproperty for seasons, episods and cast member direclty from the tv series
# GROUPING
# debuging the office, only 25 cast members?

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?seasons ?episodes ?castMember
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       #(wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P2437 ?seasons;  #--wdt:P527 ('number of seasons')---->
       wdt:P1113 ?episodes;       #--wdt:P1113 ('number of episodes')-->
       wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
       sc:name ?tvSeriesName.
}
#GROUP BY ?tvSeries ?tvSeriesName ?seasons ?episodes
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('seasons', '9'), ('episodes', '201'), ('castMember', 'http://www.wikidata.org/entity/Q1050211')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('seasons', '9'), ('episodes', '201'), ('castMember', 'http://www.wikidata.org/entity/Q1139248')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('seasons', '9'), ('episodes', '201'), ('castMember', 'http://www.wikidata.org/entity/Q216221')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('seasons', '9'), ('episodes', '201'), ('castMember', 'http://www.wikidata.org/entity/Q2238008')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('seasons', '9'), ('episodes', '201'), ('castMember', 'http://www.wikidata.org/entity/Q231203')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeri

#### Also in task 3 we found that The Office (US) has only few cast members and only in few episodes, so there is no need to go deeper

### Final query for this task

In [62]:
# Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members 
#    (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?seasons ?episodes (COUNT (?castMember) AS ?castMembers)
WHERE {   
   # bind something
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   ?tvSeries wdt:P2437 ?seasons;  #--wdt:P527 ('number of seasons')---->
       wdt:P1113 ?episodes;       #--wdt:P1113 ('number of episodes')-->
       wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
       sc:name ?tvSeriesName.
}
GROUP BY ?tvSeries ?tvSeriesName ?seasons ?episodes
ORDER BY ASC(?tvSeriesName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('seasons', '9'), ('episodes', '208'), ('castMembers', '480')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('seasons', '9'), ('episodes', '201'), ('castMembers', '25')]
2


## Task 6
Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

#### Wrong Interpretation of the KB number
I was thinking that KB number can be between any 2 actors but I found that its from only Kavin Bacon

In [63]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')
queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?castMember ?castMemberName
WHERE {   
   # bind something
   #VALUES (?tvSeries) {
   #    (wd:Q23831)  #'The Office (US)'
   #    (wd:Q147235) #'How I met your mother'
   #}
   ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName.
       
    ?castMember sc:name ?castMemberName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName', 'Everybody Hates Chris'), ('castMember', 'http://www.wikidata.org/entity/Q271464'), ('castMemberName', 'Terry Crews')]
[('tvSeries', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName', 'Everybody Hates Chris'), ('castMember', 'http://www.wikidata.org/entity/Q468012'), ('castMemberName', 'Imani Hakim')]
[('tvSeries', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName', 'Everybody Hates Chris'), ('castMember', 'http://www.wikidata.org/entity/Q456894'), ('castMemberName', 'Tichina Arnold')]
[('tvSeries', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName', 'Everybody Hates Chris'), ('castMember', 'http://www.wikidata.org/entity/Q723011'), ('castMemberName', 'Tyler James Williams')]
[('tvSeries', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName', 'Everybody Hates Chris'), ('castMember', 'http://www.wikidata.org/entity/Q948952'), ('castMemberName', 'Vincent Martella')]
[('tvSeries', '

In [66]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# this also include kbn = 1
queryString = """
SELECT DISTINCT ?tvSeries1 ?tvSeriesName1 ?castMember1 ?castMemberName1 ?tvSeries2 ?tvSeriesName2 ?castMember2 ?castMemberName2
WHERE {   
   # bind something
   #VALUES (?tvSeries) {
   #    (wd:Q23831)  #'The Office (US)'
   #    (wd:Q147235) #'How I met your mother'
   #}
   
   # A castMember in tvSeries1 with B, B castMember in tvSeries2 with C --> return A,C 
   # GET B and remove them
   
   ?tvSeries1 wdt:P161 ?castMember1;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName1.
           
    ?tvSeries2 wdt:P161 ?castMember2;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName2.
       
    ?castMember1 sc:name ?castMemberName1. 
    ?castMember2 sc:name ?castMemberName2.
    
    FILTER (?castMember1 != ?castMember2) # GET B and remove them 
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries1', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName1', 'Everybody Hates Chris'), ('castMember1', 'http://www.wikidata.org/entity/Q468012'), ('castMemberName1', 'Imani Hakim'), ('tvSeries2', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName2', 'Everybody Hates Chris'), ('castMember2', 'http://www.wikidata.org/entity/Q271464'), ('castMemberName2', 'Terry Crews')]
[('tvSeries1', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName1', 'Everybody Hates Chris'), ('castMember1', 'http://www.wikidata.org/entity/Q456894'), ('castMemberName1', 'Tichina Arnold'), ('tvSeries2', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName2', 'Everybody Hates Chris'), ('castMember2', 'http://www.wikidata.org/entity/Q271464'), ('castMemberName2', 'Terry Crews')]
[('tvSeries1', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName1', 'Everybody Hates Chris'), ('castMember1', 'http://www.wikidata.org/entity/Q723011'), ('castMemberName1', 'Tyler James Williams'), ('

In [67]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

queryString = """
SELECT DISTINCT ?tvSeries1 ?tvSeriesName1 ?castMember1 ?castMemberName1 ?tvSeries2 ?tvSeriesName2 ?castMember2 ?castMemberName2
WHERE {   
   # bind something
   #VALUES (?tvSeries) {
   #    (wd:Q23831)  #'The Office (US)'
   #    (wd:Q147235) #'How I met your mother'
   #}
   
   # A castMember in tvSeries1 with B, B castMember in tvSeries2 with C --> return A,C 
   # GET B and remove them
   
   ?tvSeries1 wdt:P161 ?castMember1;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName1.
           
    ?tvSeries2 wdt:P161 ?castMember2;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName2.
       
    ?castMember1 sc:name ?castMemberName1. 
    ?castMember2 sc:name ?castMemberName2.
    
    FILTER (?castMember1 != ?castMember2 && ?tvSeries1 != ?tvSeries2) # GET B and remove them
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries1', 'http://www.wikidata.org/entity/Q500769'), ('tvSeriesName1', 'Freaks and Geeks'), ('castMember1', 'http://www.wikidata.org/entity/Q234204'), ('castMemberName1', 'Linda Cardellini'), ('tvSeries2', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName2', 'Everybody Hates Chris'), ('castMember2', 'http://www.wikidata.org/entity/Q271464'), ('castMemberName2', 'Terry Crews')]
[('tvSeries1', 'http://www.wikidata.org/entity/Q500769'), ('tvSeriesName1', 'Freaks and Geeks'), ('castMember1', 'http://www.wikidata.org/entity/Q943613'), ('castMemberName1', 'Samm Levine'), ('tvSeries2', 'http://www.wikidata.org/entity/Q33323'), ('tvSeriesName2', 'Everybody Hates Chris'), ('castMember2', 'http://www.wikidata.org/entity/Q271464'), ('castMemberName2', 'Terry Crews')]
[('tvSeries1', 'http://www.wikidata.org/entity/Q500769'), ('tvSeriesName1', 'Freaks and Geeks'), ('castMember1', 'http://www.wikidata.org/entity/Q6184097'), ('castMemberName1', 'Jerry Messing'), ('tvSeries2', 'htt

##### Trying with graph/path patterns

In [68]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# CASE tv1, member1 with tv2, member2 -- tv1, member2 with tv2, member 1 ...

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?castMember ?castMemberName
WHERE {      
   # A castMember in tvSeries1 with B, B castMember in tvSeries2 with C --> return A,C 
   # GET B and remove them
   
   ?tvSeries wdt:P161{2} ?castMember;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName.
       
    ?castMember sc:name ?castMemberName.
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q15870780'), ('tvSeriesName', 'Gogona Gareubnidan'), ('castMember', 'http://www.wikidata.org/entity/Q4631803'), ('castMemberName', '247°F')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1353486'), ('tvSeriesName', 'Marumo no Okite'), ('castMember', 'http://www.wikidata.org/entity/Q15644319'), ('castMemberName', 'Ashita, Mama ga Inai')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1353486'), ('tvSeriesName', 'Marumo no Okite'), ('castMember', 'http://www.wikidata.org/entity/Q901021'), ('castMemberName', 'Beautiful Rain')]
[('tvSeries', 'http://www.wikidata.org/entity/Q9024286'), ('tvSeriesName', 'Los ladrones van a la oficina'), ('castMember', 'http://www.wikidata.org/entity/Q1675466'), ('castMemberName', 'Belle Époque')]
[('tvSeries', 'http://www.wikidata.org/entity/Q5801618'), ('tvSeriesName', 'Del dicho al hecho'), ('castMember', 'http://www.wikidata.org/entity/Q1675466'), ('castMemberName', 'Belle Époque')]
[('tvSeries', 'htt

In [69]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# TRYING WITH GRAPH PATTERNS

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?castMember ?castMemberName
WHERE {      
   # A castMember in tvSeries1 with B, B castMember in tvSeries2 with C --> return A,C 
   # GET B and remove them
   
   # A - C
   {
       ?tvSeries1 wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
           wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
           sc:name ?tvSeriesName1.
   }
   MINUS
   {
       ?tvSeries2 wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
           wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
           sc:name ?tvSeriesName2.
   }
   
    ?castMember sc:name ?castMemberName.
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


#### Correct interpretation
Studied more Kavin Bacon number and found the correct interpretation

##### Finding Kavin Bacon

In [70]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?castMember ?castMemberName
WHERE {      
   #VALUES (?tvSeries) {
   #    (wd:Q23831)  #'The Office (US)'
   #    (wd:Q147235) #'How I met your mother'
   #}
   
   ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName. 
   
    ?castMember sc:name ?castMemberName.
    
    FILTER (regex(?castMemberName, "Bacon"))
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q91628404'), ('tvSeriesName', 'White Lines'), ('castMember', 'http://www.wikidata.org/entity/Q87070070'), ('castMemberName', 'Jamie Bacon')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1189631'), ('tvSeriesName', 'The Following'), ('castMember', 'http://www.wikidata.org/entity/Q3454165'), ('castMemberName', 'Kevin Bacon')]
[('tvSeries', 'http://www.wikidata.org/entity/Q48742976'), ('tvSeriesName', 'City on a Hill'), ('castMember', 'http://www.wikidata.org/entity/Q3454165'), ('castMemberName', 'Kevin Bacon')]
3


#### Finding how to link Kavin Bacon with the two tv series: KBacon number 1

In [71]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName
WHERE {      
   #VALUES (?tvSeries) {
   #    (wd:Q23831)  #'The Office (US)'
   #    (wd:Q147235) #'How I met your mother'
   #}
   
   
   ?tvSeries wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')---------->wd:Q3454165 ('Kevin Bacon')
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName.    
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q1189631'), ('tvSeriesName', 'The Following')]
[('tvSeries', 'http://www.wikidata.org/entity/Q48742976'), ('tvSeriesName', 'City on a Hill')]
2


In [72]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName
WHERE {      
   #VALUES (?tvSeries) {
   #    (wd:Q23831)  #'The Office (US)'
   #    (wd:Q147235) #'How I met your mother'
   #}
   
   
   ?tvSeries wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')---------->wd:Q3454165 ('Kevin Bacon')
       sc:name ?tvSeriesName.    
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q122113'), ('tvSeriesName', 'Super')]
[('tvSeries', 'http://www.wikidata.org/entity/Q463927'), ('tvSeriesName', 'Diner')]
[('tvSeries', 'http://www.wikidata.org/entity/Q636062'), ('tvSeriesName', 'Loverboy')]
[('tvSeries', 'http://www.wikidata.org/entity/Q691672'), ('tvSeriesName', 'Frost/Nixon')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1917925'), ('tvSeriesName', 'My Dog Skip')]
[('tvSeries', 'http://www.wikidata.org/entity/Q5265117'), ('tvSeriesName', 'Destination Anywhere (film)')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1154802'), ('tvSeriesName', 'Murder in the First')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1243029'), ('tvSeriesName', 'Friday the 13th')]
[('tvSeries', 'http://www.wikidata.org/entity/Q108946'), ('tvSeriesName', 'A Few Good Men')]
[('tvSeries', 'http://www.wikidata.org/entity/Q223596'), ('tvSeriesName', 'X-Men: First Class')]
[('tvSeries', 'http://www.wikidata.org/entity/Q339045'), ('tvSeri

In [73]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1

queryString = """
SELECT DISTINCT ?filmOrSeriesOfKevin ?filmOrSeriesOfKevinName ?castMember ?castMemberName ?tvSeriesName
WHERE {      
   VALUES (?tvSeries) {
       (wd:Q23831)  #'The Office (US)'
       (wd:Q147235) #'How I met your mother'
   }
   
   ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')---------->
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       sc:name ?tvSeriesName. 
   
   ?castMember sc:name ?castMemberName.
      
   ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')---------->wd:Q3454165 ('Kevin Bacon')
       sc:name ?filmOrSeriesOfKevinName.    
       
       
   # KBnumber = 1
   ?filmOrSeriesOfKevin wdt:P161 ?castMember.
   
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('filmOrSeriesOfKevin', 'http://www.wikidata.org/entity/Q19824612'), ('filmOrSeriesOfKevinName', 'Cop Car'), ('castMember', 'http://www.wikidata.org/entity/Q234514'), ('castMemberName', 'Camryn Manheim'), ('tvSeriesName', 'How I Met Your Mother')]
[('filmOrSeriesOfKevin', 'http://www.wikidata.org/entity/Q16167570'), ('filmOrSeriesOfKevinName', 'Skum Rocks!'), ('castMember', 'http://www.wikidata.org/entity/Q234715'), ('castMemberName', 'Jamie-Lynn Sigler'), ('tvSeriesName', 'How I Met Your Mother')]
[('filmOrSeriesOfKevin', 'http://www.wikidata.org/entity/Q975358'), ('filmOrSeriesOfKevinName', 'Stir of Echoes'), ('castMember', 'http://www.wikidata.org/entity/Q199929'), ('castMemberName', 'Jennifer Morrison'), ('tvSeriesName', 'How I Met Your Mother')]
[('filmOrSeriesOfKevin', 'http://www.wikidata.org/entity/Q17508638'), ('filmOrSeriesOfKevinName', 'The Darkness'), ('castMember', 'http://www.wikidata.org/entity/Q199929'), ('castMemberName', 'Jennifer Morrison'), ('tvSeriesName',

#### Assumptions: 
We are taking into account only the two tv series (as in the task is written with "the") 
and now we are sure to have link between Kevin Bacon and both tv series (prev query)

#### KB1 found, *wrt tv series

In [76]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2

queryString = """
SELECT DISTINCT *
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
    ?castMember sc:name ?castMemberName.
    
    FILTER (?innerCastMember = ?castMember)
    {
        SELECT DISTINCT ?innerCastMember
        WHERE {
           VALUES (?tvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?tvSeries wdt:P161 ?innerCastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?innerCastMember.
        }
    }
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('castMember', 'http://www.wikidata.org/entity/Q234514'), ('tvSeriesName', 'How I Met Your Mother'), ('castMemberName', 'Camryn Manheim'), ('innerCastMember', 'http://www.wikidata.org/entity/Q234514')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('castMember', 'http://www.wikidata.org/entity/Q234715'), ('tvSeriesName', 'How I Met Your Mother'), ('castMemberName', 'Jamie-Lynn Sigler'), ('innerCastMember', 'http://www.wikidata.org/entity/Q234715')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('castMember', 'http://www.wikidata.org/entity/Q199929'), ('tvSeriesName', 'How I Met Your Mother'), ('castMemberName', 'Jennifer Morrison'), ('innerCastMember', 'http://www.wikidata.org/entity/Q199929')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('castMember', 'http://www.wikidata.org/entity/Q312705'), ('tvSeriesName', 'How I Met Your Mother'), ('castMemberName', 'John Cho'), ('innerCastMember',

#### Finding KB2

In [78]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result

queryString = """
SELECT DISTINCT ?castMember ?castMemberName ?innerCastMember ?innerCastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?castMember sc:name ?castMemberName.
    
    FILTER (?castMember != ?innerCastMember)
    {
        SELECT DISTINCT ?innerCastMember ?innerCastMemberName
        WHERE {
           VALUES (?tvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?tvSeries wdt:P161 ?innerCastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?innerCastMember sc:name ?innerCastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?innerCastMember.
        }
    }
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q269891'), ('innerCastMemberName', 'Julianna Guill')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q312705'), ('innerCastMemberName', 'John Cho')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q234715'), ('innerCastMemberName', 'Jamie-Lynn Sigler')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q311271'), ('innerCastMemberName', 'John Lithgow')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q234137'), ('innerCas

##### Prob: Joining with both tvSeries gives also wrong combinations

In [79]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries

queryString = """
SELECT DISTINCT ?castMember ?castMemberName ?innerCastMember ?innerCastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?castMember sc:name ?castMemberName.
    
    FILTER (?castMember != ?innerCastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?innerCastMember ?innerCastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?innerCastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?innerCastMember sc:name ?innerCastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?innerCastMember.
        }
    }
}
ORDER BY (?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q312705'), ('innerCastMemberName', 'John Cho')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q311271'), ('innerCastMemberName', 'John Lithgow')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q269891'), ('innerCastMemberName', 'Julianna Guill')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q234137'), ('innerCastMemberName', 'Megan Mullally')]
[('castMember', 'http://www.wikidata.org/entity/Q8349'), ('castMemberName', '"Weird Al" Yankovic'), ('innerCastMember', 'http://www.wikidata.org/entity/Q234715'), ('innerCastMe

In [80]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries

queryString = """
SELECT DISTINCT ?tvSeriesName ?castMemberName ?innerCastMemberName 
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?castMember sc:name ?castMemberName.
    
    FILTER (?castMember != ?innerCastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?innerCastMember ?innerCastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?innerCastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?innerCastMember sc:name ?innerCastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?innerCastMember.
        }
    }
}
ORDER BY DESC (?tvSeriesName) ASC(?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeriesName', 'The Office'), ('castMemberName', 'Amy Ryan'), ('innerCastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'Amy Ryan'), ('innerCastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'Angela Kinsey'), ('innerCastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'Angela Kinsey'), ('innerCastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'B. J. Novak'), ('innerCastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'B. J. Novak'), ('innerCastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'Brian Baumgartner'), ('innerCastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'Brian Baumgartner'), ('innerCastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('castMemberName', 'Catherine Tate'), ('innerCastMemberName', 'Steve Carell')

In [81]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming

queryString = """
SELECT DISTINCT ?tvSeriesName ?bacon2CastMemberName ?bacon1CastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY DESC (?tvSeriesName) ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Amy Ryan'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Amy Ryan'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Angela Kinsey'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Angela Kinsey'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'B. J. Novak'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'B. J. Novak'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Brian Baumgartner'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Brian Baumgartner'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberNam

In [82]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming
# extending limit

queryString = """
SELECT DISTINCT ?tvSeriesName ?bacon2CastMemberName ?bacon1CastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY DESC (?tvSeriesName) ASC(?bacon2CastMemberName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Amy Ryan'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Amy Ryan'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Angela Kinsey'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Angela Kinsey'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'B. J. Novak'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'B. J. Novak'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Brian Baumgartner'), ('bacon1CastMemberName', 'Steve Carell')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberName', 'Brian Baumgartner'), ('bacon1CastMemberName', 'Rainn Wilson')]
[('tvSeriesName', 'The Office'), ('bacon2CastMemberNam

In [83]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming
# extending limit

queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName ?bacon1CastMember ?bacon1CastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q312705'), ('bacon1CastMemberName', 'John Cho')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q311271'), ('bacon1CastMemberName', 'John Lithgow')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q269891'), ('bacon1CastMemberName', 'Julianna Guill')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q234137'), ('bacon1CastMemberName', 'Megan Mullally')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1

##### Solved 

#### Counting 

In [86]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming
# extending limit
# COUNTING

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) as ?actors)
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('actors', '16')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('actors', '2')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('actors', '15')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('actors', '1')]
4


In [87]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming
# extending limit
# COUNTING
# removed wrong order by

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) as ?actors)
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('actors', '48')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('actors', '7664')]
2


##### Prob: To many actors

In [88]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming
# extending limit
# COUNTING ONLY KB2 + DISTINCT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(DISTINCT ?bacon2CastMember) as ?bacon2CastMembers)
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('bacon2CastMembers', '480')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('bacon2CastMembers', '25')]
2


In [89]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming
# extending limit
# COUNTING ONLY KB2 + DISTINCT
# PROB: STILL HAVE 25 (max cast member)

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(DISTINCT ?bacon2CastMember) as ?bacon2CastMembers)
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('bacon2CastMembers', '480')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('bacon2CastMembers', '25')]
2


##### DEBUGGING started

In [90]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob to much count


queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q465556'), ('bacon2CastMemberName', 'Annie Ilonzeh')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q269891'), ('bacon2CastMemberName', 'Julianna Guill')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q4888924'), ('bacon2CastMemberName', 'Benjamin Koldyke')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q446031'), ('bacon2CastMemberName', 'Nikki Griffin')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q312705'), ('bacon2CastMemberName', 'John Cho')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q469579'), ('bacon2CastMemberName', 'Mircea Monroe')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q516659'), ('bacon2CastMemberName', 'Virginia Williams')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q522856'), ('bacon2CastMemberName', 'Kate Micucci')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q200566'), ('bacon2CastMemberName', 'Cobie Smulders')]
[('bacon2CastMem

In [91]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob to much count
# + ORDER BY


queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q24576840'), ('bacon2CastMemberName', 'Aaron Hendry')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q667979'), ('bacon2CastMemberName', 'Aaron Hill')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q2708237'), ('bacon2CastMemberName', 'Abby Elliott')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q50736659'), ('bacon2CastMemberName', 'Abhi Sinha')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q321131'), ('bacon2CastMemberName', 'Abigail Spencer')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q50736705'), ('bacon2CastMemberName', 'Adam Paul')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q151866'), ('bacon2CastMemberName', 'Adriana Lima')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q2827706'), ('bacon2CastMemberName', "Ahna O'Reilly")]
[('bacon2CastMember

In [93]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob to much count
# + ORDER BY x2


queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName ?tvSeriesName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY DESC(?tvSeriesName) ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q231203'), ('bacon2CastMemberName', 'Amy Ryan'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q2669971'), ('bacon2CastMemberName', 'Angela Kinsey'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q459925'), ('bacon2CastMemberName', 'B. J. Novak'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q2924850'), ('bacon2CastMemberName', 'Brian Baumgartner'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q254766'), ('bacon2CastMemberName', 'Catherine Tate'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q512818'), ('bacon2CastMemberName', 'Clark Duke'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q926912'), ('bacon2CastMemberName', 'Craig Robinson'), ('tvSeriesName', 'The Office')]
[('bacon2CastM

In [ ]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob to much count
# + ORDER BY x2
# oly KB1, by changing FILTER


queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName ?tvSeriesName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember = ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY DESC(?tvSeriesName) ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q349548'), ('bacon2CastMemberName', 'Rainn Wilson'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q216221'), ('bacon2CastMemberName', 'Steve Carell'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q234514'), ('bacon2CastMemberName', 'Camryn Manheim'), ('tvSeriesName', 'How I Met Your Mother')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q234715'), ('bacon2CastMemberName', 'Jamie-Lynn Sigler'), ('tvSeriesName', 'How I Met Your Mother')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q199929'), ('bacon2CastMemberName', 'Jennifer Morrison'), ('tvSeriesName', 'How I Met Your Mother')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q312705'), ('bacon2CastMemberName', 'John Cho'), ('tvSeriesName', 'How I Met Your Mother')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q311271'), ('bacon2CastMemberName', 'John Lithgow'),

In [95]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob: to much count
# debugging only The Office


queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName ?tvSeriesName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               #(wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY DESC(?tvSeriesName) ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q231203'), ('bacon2CastMemberName', 'Amy Ryan'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q2669971'), ('bacon2CastMemberName', 'Angela Kinsey'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q459925'), ('bacon2CastMemberName', 'B. J. Novak'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q2924850'), ('bacon2CastMemberName', 'Brian Baumgartner'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q254766'), ('bacon2CastMemberName', 'Catherine Tate'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q512818'), ('bacon2CastMemberName', 'Clark Duke'), ('tvSeriesName', 'The Office')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q926912'), ('bacon2CastMemberName', 'Craig Robinson'), ('tvSeriesName', 'The Office')]
[('bacon2CastM

##### Real prob found: KB2 containing also KB1

##### Finding solution to remove KB1 from KB2

In [96]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob: to much count
# debugging only The Office


queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName ?tvSeriesName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    MINUS 
    #FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               #(wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY DESC(?tvSeriesName) ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [97]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob: to much count
# debugging only The Office


queryString = """
SELECT DISTINCT ?castMember ?castMemberName ?tvSeriesName
WHERE {      
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE { 
            VALUES (?tvSeries) {
                (wd:Q23831)  #'The Office (US)'
                #(wd:Q147235) #'How I met your mother'
            }

            ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
                wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
                sc:name ?tvSeriesName. 

            ?castMember sc:name ?castMemberName.
        }
    }
    MINUS 
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE {
           VALUES (?tvSeries) {
               (wd:Q23831)  #'The Office (US)'
               #(wd:Q147235) #'How I met your mother'
           }

           ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
               sc:name ?tvSeriesName. 

           ?castMember sc:name ?castMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?castMember.
        }
    }
}
ORDER BY DESC(?tvSeriesName) ASC(?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q231203'), ('castMemberName', 'Amy Ryan'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q2669971'), ('castMemberName', 'Angela Kinsey'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q459925'), ('castMemberName', 'B. J. Novak'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q2924850'), ('castMemberName', 'Brian Baumgartner'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q254766'), ('castMemberName', 'Catherine Tate'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q512818'), ('castMemberName', 'Clark Duke'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q926912'), ('castMemberName', 'Craig Robinson'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q2238008'), ('castMemberName', 'Creed Bratton'), (

##### END DEBUGGING. 
##### Possible solution found, OK now:
- Q349548 Rainn Wilson
- Q216221 Steve Carell

are removed

In [98]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# back to counting


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(DISTINCT ?castMember) as ?bacon2CastMembers)
WHERE {      
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE { 
            VALUES (?tvSeries) {
                (wd:Q23831)  #'The Office (US)'
                (wd:Q147235) #'How I met your mother'
            }

            ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
                wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
                sc:name ?tvSeriesName. 

            ?castMember sc:name ?castMemberName.
        }
    }
    MINUS # BACON NUMBER 1
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE {
           VALUES (?tvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
               sc:name ?tvSeriesName. 

           ?castMember sc:name ?castMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?castMember.
        }
    }
}
ORDER BY DESC(?tvSeriesName) ASC(?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('bacon2CastMembers', '1')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('bacon2CastMembers', '1')]
2


In [99]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# back to counting


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) as ?bacon2CastMembers)
WHERE {      
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE { 
            VALUES (?tvSeries) {
                (wd:Q23831)  #'The Office (US)'
                (wd:Q147235) #'How I met your mother'
            }

            ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
                wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
                sc:name ?tvSeriesName. 

            ?castMember sc:name ?castMemberName.
        }
    }
    MINUS # BACON NUMBER 1
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE {
           VALUES (?tvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
               sc:name ?tvSeriesName. 

           ?castMember sc:name ?castMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?castMember.
        }
    }
}
ORDER BY DESC(?tvSeriesName) ASC(?castMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('bacon2CastMembers', '1')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('bacon2CastMembers', '1')]
2


#### Possible solution found

In [101]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# back to counting


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) as ?bacon2CastMembers)
WHERE {      
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE { 
            VALUES (?tvSeries) {
                (wd:Q23831)  #'The Office (US)'
                (wd:Q147235) #'How I met your mother'
            }

            ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
                wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
                sc:name ?tvSeriesName. 

            ?castMember sc:name ?castMemberName.
        }
    }
    MINUS # BACON NUMBER 1
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE {
           VALUES (?tvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
               sc:name ?tvSeriesName. 

           ?castMember sc:name ?castMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?castMember.
        }
    }
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('bacon2CastMembers', '464')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('bacon2CastMembers', '23')]
2


#### Trying to solve with path patterns

In [2]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:161/wdt:161/^wdt:161/wdt:161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [3]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    ?castMember ^wdt:161/wdt:161/^wdt:161/wdt:161 wd:Q3454165;
        sc:name ?castMemberName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [4]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns

queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    ?castMember ^wdt:161/wdt:161 wd:Q3454165;
        sc:name ?castMemberName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    ?castMember wdt:161/wdt:161 wd:Q3454165;
        sc:name ?castMemberName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [6]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    ?castMember ^wdt:P161/wdt:P161/^wdt:P161/wdt:P161 wd:Q3454165;
        sc:name ?castMemberName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q15867224'), ('castMemberName', "Mark O'Brien")]
[('castMember', 'http://www.wikidata.org/entity/Q1354565'), ('castMemberName', 'Julian Wadham')]
[('castMember', 'http://www.wikidata.org/entity/Q18162913'), ('castMemberName', 'Callum Turner')]
[('castMember', 'http://www.wikidata.org/entity/Q17100475'), ('castMemberName', 'Brian Kubach')]
[('castMember', 'http://www.wikidata.org/entity/Q266888'), ('castMemberName', 'Vinessa Shaw')]
[('castMember', 'http://www.wikidata.org/entity/Q269891'), ('castMemberName', 'Julianna Guill')]
[('castMember', 'http://www.wikidata.org/entity/Q391359'), ('castMemberName', 'David Harbour')]
[('castMember', 'http://www.wikidata.org/entity/Q5799862'), ('castMemberName', 'Josh Helman')]
[('castMember', 'http://www.wikidata.org/entity/Q2151703'), ('castMemberName', 'James Murtaugh')]
[('castMember', 'http://www.wikidata.org/entity/Q6758083'), ('castMemberName', 'Marcus Carl Franklin')]
10


In [7]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161/^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q231203'), ('castMemberName', 'Amy Ryan')]
[('castMember', 'http://www.wikidata.org/entity/Q238877'), ('castMemberName', 'Jenna Fischer')]
[('castMember', 'http://www.wikidata.org/entity/Q269901'), ('castMemberName', 'Melora Hardin')]
[('castMember', 'http://www.wikidata.org/entity/Q328790'), ('castMemberName', 'Ed Helms')]
[('castMember', 'http://www.wikidata.org/entity/Q508280'), ('castMemberName', 'Phyllis Smith')]
[('castMember', 'http://www.wikidata.org/entity/Q539917'), ('castMemberName', 'Mindy Kaling')]
[('castMember', 'http://www.wikidata.org/entity/Q64560'), ('castMemberName', 'David Denman')]
[('castMember', 'http://www.wikidata.org/entity/Q926912'), ('castMemberName', 'Craig Robinson')]
[('castMember', 'http://www.wikidata.org/entity/Q296928'), ('castMemberName', 'James Spader')]
[('castMember', 'http://www.wikidata.org/entity/Q1139248'), ('castMemberName', 'Oscar Nunez')]
[('castMember', 'http://www.wikidata.org/entit

In [9]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q296928'), ('castMemberName', 'James Spader')]
[('castMember', 'http://www.wikidata.org/entity/Q926912'), ('castMemberName', 'Craig Robinson')]
[('castMember', 'http://www.wikidata.org/entity/Q238877'), ('castMemberName', 'Jenna Fischer')]
[('castMember', 'http://www.wikidata.org/entity/Q508280'), ('castMemberName', 'Phyllis Smith')]
[('castMember', 'http://www.wikidata.org/entity/Q231203'), ('castMemberName', 'Amy Ryan')]
[('castMember', 'http://www.wikidata.org/entity/Q269901'), ('castMemberName', 'Melora Hardin')]
[('castMember', 'http://www.wikidata.org/entity/Q64560'), ('castMemberName', 'David Denman')]
[('castMember', 'http://www.wikidata.org/entity/Q328790'), ('castMemberName', 'Ed Helms')]
[('castMember', 'http://www.wikidata.org/entity/Q539917'), ('castMemberName', 'Mindy Kaling')]
[('castMember', 'http://www.wikidata.org/entity/Q216221'), ('castMemberName', 'Steve Carell')]
[('castMember', 'http://www.wikidata.org/entit

In [10]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?KB1 ?KB1Name
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('KB1', 'http://www.wikidata.org/entity/Q451830'), ('KB1Name', 'Lilyan Chauvin')]
[('KB1', 'http://www.wikidata.org/entity/Q19609853'), ('KB1Name', 'Eloy Casados')]
[('KB1', 'http://www.wikidata.org/entity/Q269891'), ('KB1Name', 'Julianna Guill')]
[('KB1', 'http://www.wikidata.org/entity/Q3097804'), ('KB1Name', 'John Bennett Perry')]
[('KB1', 'http://www.wikidata.org/entity/Q164328'), ('KB1Name', 'David Koechner')]
[('KB1', 'http://www.wikidata.org/entity/Q189400'), ('KB1Name', 'Brooke Shields')]
[('KB1', 'http://www.wikidata.org/entity/Q391359'), ('KB1Name', 'David Harbour')]
[('KB1', 'http://www.wikidata.org/entity/Q434291'), ('KB1Name', 'Rade Šerbedžija')]
[('KB1', 'http://www.wikidata.org/entity/Q1390515'), ('KB1Name', 'Richard Herd')]
[('KB1', 'http://www.wikidata.org/entity/Q1101612'), ('KB1Name', 'Clint Howard')]
[('KB1', 'http://www.wikidata.org/entity/Q198684'), ('KB1Name', 'Wayne Duvall')]
[('KB1', 'http://www.wikidata.org/entity/Q3376428'), ('KB1Name', 'Peter Appel'

In [11]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns


queryString = """
SELECT DISTINCT ?KB1 ?KB1Name
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('KB1', 'http://www.wikidata.org/entity/Q216221'), ('KB1Name', 'Steve Carell')]
[('KB1', 'http://www.wikidata.org/entity/Q349548'), ('KB1Name', 'Rainn Wilson')]
2


In [12]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2


queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    FILTER (?castMember != ?KB1)
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q1050211'), ('castMemberName', 'Leslie David Baker')]
[('castMember', 'http://www.wikidata.org/entity/Q1139248'), ('castMemberName', 'Oscar Nunez')]
[('castMember', 'http://www.wikidata.org/entity/Q2238008'), ('castMemberName', 'Creed Bratton')]
[('castMember', 'http://www.wikidata.org/entity/Q231203'), ('castMemberName', 'Amy Ryan')]
[('castMember', 'http://www.wikidata.org/entity/Q238877'), ('castMemberName', 'Jenna Fischer')]
[('castMember', 'http://www.wikidata.org/entity/Q254766'), ('castMemberName', 'Catherine Tate')]
[('castMember', 'http://www.wikidata.org/entity/Q2669971'), ('castMemberName', 'Angela Kinsey')]
[('castMember', 'http://www.wikidata.org/entity/Q2671438'), ('castMemberName', 'Paul Lieberstein')]
[('castMember', 'http://www.wikidata.org/entity/Q269901'), ('castMemberName', 'Melora Hardin')]
[('castMember', 'http://www.wikidata.org/entity/Q2924850'), ('castMemberName', 'Brian Baumgartner')]
[('castMember', 'htt

In [20]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?castMember ?castMemberName (GROUP_CONCAT(?KBNumber;SEPARATOR=",") as ?KB)
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    BIND(IF(?KB1 = ?castMember, 2, 1) AS ?KBNumber)
}
GROUP BY ?castMember ?castMemberName
#HAVING (COUNT(*) = 1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q72077'), ('castMemberName', 'Ellie Kemper'), ('KB', '1,1,1')]
[('castMember', 'http://www.wikidata.org/entity/Q216221'), ('castMemberName', 'Steve Carell'), ('KB', '2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2')]
[('castMember', 'http://www.wikidata.org/entity/Q2924850'), ('castMemberName', 'Brian Baumgartner'), ('KB', '1,1')]
[('castMember', 'http://www.wikidata.org/entity/Q2238008'), ('castMemberName', 'Creed Bratton'), ('KB', '1,1')]
[('castMember', 'http://www.wikidata.org/entity/Q6124807'), ('castMemberName', 'Jake Lacy'), ('KB', '1,1')]
[('castMember', 'http://www.wikidata.org/entity/Q269901'), ('castMemberName', 'Melora Hardin'), ('KB', '1,1')]
[('castMember', 'http://www.wikidata.org/entity/Q3028200'), ('castMemberName', 'Kate Flannery'), ('KB', '1,1')]
[('castMember', 'http://www.wikidata.org/entity/Q2669971'), ('castMemberName', 'Angela Kinsey'), ('KB', '1,1')]
[('castMember'

In [21]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?castMember ?castMemberName MAX(?KBNumber) as ?KB
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    BIND(IF(?KB1 = ?castMember, 2, 1) AS ?KBNumber)
}
GROUP BY ?castMember ?castMemberName
#HAVING (COUNT(*) = 1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q926912'), ('castMemberName', 'Craig Robinson'), ('KB', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q2924850'), ('castMemberName', 'Brian Baumgartner'), ('KB', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q6124807'), ('castMemberName', 'Jake Lacy'), ('KB', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q349548'), ('castMemberName', 'Rainn Wilson'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q459925'), ('castMemberName', 'B. J. Novak'), ('KB', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q1139248'), ('castMemberName', 'Oscar Nunez'), ('KB', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q2671438'), ('castMemberName', 'Paul Lieberstein'), ('KB', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q296928'), ('castMemberName', 'James Spader'), ('KB', '1')]
[('castMember', 'http://www.wikidata.org/entity/Q64560'), ('castMemberName', 'David Denman'), ('KB', '1')]
[('castMemb

In [22]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?castMember ?castMemberName MAX(?KBNumber) as ?KB
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    BIND(IF(?KB1 = ?castMember, 2, 1) AS ?KBNumber)
}
GROUP BY ?castMember ?castMemberName
HAVING (MAX(?KBNumber) >  1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q349548'), ('castMemberName', 'Rainn Wilson'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q216221'), ('castMemberName', 'Steve Carell'), ('KB', '2')]
2


In [23]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1
# CORRECTED KB NUMBER, INVERTING IN THE BIND


queryString = """
SELECT DISTINCT ?castMember ?castMemberName MAX(?KBNumber) as ?KB
WHERE {

    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    BIND(IF(?KB1 = ?castMember, 1, 2) AS ?KBNumber)
}
GROUP BY ?castMember ?castMemberName
HAVING (MAX(?KBNumber) >  1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q926912'), ('castMemberName', 'Craig Robinson'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q2924850'), ('castMemberName', 'Brian Baumgartner'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q6124807'), ('castMemberName', 'Jake Lacy'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q349548'), ('castMemberName', 'Rainn Wilson'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q459925'), ('castMemberName', 'B. J. Novak'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q1139248'), ('castMemberName', 'Oscar Nunez'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q2671438'), ('castMemberName', 'Paul Lieberstein'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q296928'), ('castMemberName', 'James Spader'), ('KB', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q64560'), ('castMemberName', 'David Denman'), ('KB', '2')]
[('castMemb

In [25]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName #MIN(?KBNumber) as ?KB
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    BIND(IF(?KB1 = ?castMember, 1, 2) AS ?KBNumber)
}
GROUP BY ?castMember ?castMemberName ?tvSeries ?tvSeriesName
HAVING (MIN(?KBNumber) >  1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q459925'), ('castMemberName', 'B. J. Novak'), ('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q231203'), ('castMemberName', 'Amy Ryan'), ('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q269901'), ('castMemberName', 'Melora Hardin'), ('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q926912'), ('castMemberName', 'Craig Robinson'), ('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q2671438'), ('castMemberName', 'Paul Lieberstein'), ('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office')]
[('castMember', 'http://www.wikidata.org/entity/Q2924850'), ('castMemberName', 

In [27]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(?castMember) AS ?kb2Members)#MIN(?KBNumber) as ?KB
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    BIND(IF(?KB1 = ?castMember, 1, 2) AS ?KBNumber)
}
GROUP BY ?tvSeries ?tvSeriesName
HAVING (MIN(?KBNumber) >  1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [28]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName #MIN(?KBNumber) as ?KB
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }
    
    ?castMember ^wdt:P161/wdt:P161 ?KB1;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?castMemberName.
        
    ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
        sc:name ?KB1Name.
    
    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
        
    BIND(IF(?KB1 = ?castMember, 1, 2) AS ?KBNumber)
}
GROUP BY ?tvSeries ?tvSeriesName
HAVING (MIN(?KBNumber) >  1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [29]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) AS ?kb2Members)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName #MIN(?KBNumber) as ?KB
    WHERE {
        VALUES (?tvSeries) {
            (wd:Q23831)  #'The Office (US)'
            #(wd:Q147235) #'How I met your mother'
        }

        ?castMember ^wdt:P161/wdt:P161 ?KB1;
            ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
            sc:name ?castMemberName.

        ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
            ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
            sc:name ?KB1Name.

        ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
            sc:name ?tvSeriesName.

        BIND(IF(?KB1 = ?castMember, 1, 2) AS ?KBNumber)
    }
    GROUP BY ?castMember ?castMemberName ?tvSeries ?tvSeriesName
    HAVING (MIN(?KBNumber) >  1)
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('kb2Members', '23')]
1


#### DONE ALSO WITH PATH PATTERNS

In [30]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Trying with path patterns
# OK KB1
# TRYING KB2, removing KB1


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) AS ?kb2Members)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName #MIN(?KBNumber) as ?KB
    WHERE {
        VALUES (?tvSeries) {
            (wd:Q23831)  #'The Office (US)'
            (wd:Q147235) #'How I met your mother'
        }

        ?castMember ^wdt:P161/wdt:P161 ?KB1;
            ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
            sc:name ?castMemberName.

        ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;
            ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')-->
            sc:name ?KB1Name.

        ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
            sc:name ?tvSeriesName.

        BIND(IF(?KB1 = ?castMember, 1, 2) AS ?KBNumber)
    }
    GROUP BY ?castMember ?castMemberName ?tvSeries ?tvSeriesName
    HAVING (MIN(?KBNumber) >  1)
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('kb2Members', '464')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('kb2Members', '23')]
2


#### OK with path patterns + BINDING
#### TRYING OTHER WAYS/PATTERNS

In [32]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }

    ?castMember ^wdt:P161/wdt:P161{2} wd:Q3454165;       #?KB1 <--wdt:P161 ('cast member')--[]--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')--> (wd:Q23831)|(wd:Q147235)
        sc:name ?KB1Name.

    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q216221')]
[('castMember', 'http://www.wikidata.org/entity/Q349548')]
2


In [33]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }

    ?castMember ^wdt:P161/wdt:P161{2} wd:Q3454165;       #?KB1 <--wdt:P161 ('cast member')--[]--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')--> (wd:Q23831)|(wd:Q147235)
        sc:name ?castMemberName.
        

    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q216221'), ('castMemberName', 'Steve Carell')]
[('castMember', 'http://www.wikidata.org/entity/Q349548'), ('castMemberName', 'Rainn Wilson')]
2


In [34]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }

    ?castMember wdt:P161{4} wd:Q3454165;       #?KB1 <--wdt:P161 ('cast member')--[]--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')--> (wd:Q23831)|(wd:Q147235)
        sc:name ?castMemberName.
        

    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [35]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }

    ?castMember wdt:P161{2} wd:Q3454165;       #?KB1 <--wdt:P161 ('cast member')--[]--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')--> (wd:Q23831)|(wd:Q147235)
        sc:name ?castMemberName.
        

    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [36]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        #(wd:Q147235) #'How I met your mother'
    }

    ?castMember (^wdt:P161/wdt:P161){2} wd:Q3454165;       #?KB1 <--wdt:P161 ('cast member')--[]--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
        ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')--> (wd:Q23831)|(wd:Q147235)
        sc:name ?castMemberName.
        

    ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q1050211'), ('castMemberName', 'Leslie David Baker')]
[('castMember', 'http://www.wikidata.org/entity/Q1139248'), ('castMemberName', 'Oscar Nunez')]
[('castMember', 'http://www.wikidata.org/entity/Q216221'), ('castMemberName', 'Steve Carell')]
[('castMember', 'http://www.wikidata.org/entity/Q2238008'), ('castMemberName', 'Creed Bratton')]
[('castMember', 'http://www.wikidata.org/entity/Q231203'), ('castMemberName', 'Amy Ryan')]
[('castMember', 'http://www.wikidata.org/entity/Q238877'), ('castMemberName', 'Jenna Fischer')]
[('castMember', 'http://www.wikidata.org/entity/Q254766'), ('castMemberName', 'Catherine Tate')]
[('castMember', 'http://www.wikidata.org/entity/Q2669971'), ('castMemberName', 'Angela Kinsey')]
[('castMember', 'http://www.wikidata.org/entity/Q2671438'), ('castMemberName', 'Paul Lieberstein')]
[('castMember', 'http://www.wikidata.org/entity/Q269901'), ('castMemberName', 'Melora Hardin')]
[('castMember', 'http://ww

### Final query for this task

#### Interpretation 1: return a list of couples actor/actress IRI and label
Returning all the couples of (KB2,KB1)

In [83]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# The Bacon number of an actor is the number of degrees of separation he or she has from Bacon, as defined by the game.
# FIND Kevin Bacon's tv series or film
# Bacon number = 1
# Kevin --> film <-- actor1 --> HIMYM <-- actor2 == result
# solving wrong matchings, between tvSeries
# renaming
# extending limit

queryString = """
SELECT DISTINCT ?bacon2CastMember ?bacon2CastMemberName ?bacon1CastMember ?bacon1CastMemberName
WHERE {      
    VALUES (?tvSeries) {
        (wd:Q23831)  #'The Office (US)'
        (wd:Q147235) #'How I met your mother'
    }

    ?tvSeries wdt:P161 ?bacon2CastMember;      #--wdt:P161 ('cast member')-->
        wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
        sc:name ?tvSeriesName. 
        
    ?bacon2CastMember sc:name ?bacon2CastMemberName.
    
    FILTER (?bacon2CastMember != ?bacon1CastMember && ?innerTvSeries = ?tvSeries)
    {
        SELECT DISTINCT ?bacon1CastMember ?bacon1CastMemberName ?innerTvSeries
        WHERE {
           VALUES (?innerTvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?innerTvSeries wdt:P161 ?bacon1CastMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426.             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')

           ?bacon1CastMember sc:name ?bacon1CastMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?bacon1CastMember.
        }
    }
}
ORDER BY ASC(?bacon2CastMemberName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q312705'), ('bacon1CastMemberName', 'John Cho')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q311271'), ('bacon1CastMemberName', 'John Lithgow')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q269891'), ('bacon1CastMemberName', 'Julianna Guill')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1CastMember', 'http://www.wikidata.org/entity/Q234137'), ('bacon1CastMemberName', 'Megan Mullally')]
[('bacon2CastMember', 'http://www.wikidata.org/entity/Q8349'), ('bacon2CastMemberName', '"Weird Al" Yankovic'), ('bacon1

#### Interpretation 2: return how many of the actors who are member of the cast of the tv series ...
Returning only the count of KB2 cast members:
1) The Office (US) has in total of 25 cast member and 2 are KB1 and the remaining 23 are KB2
1) HIMYM has in total 480 cast member and 16 are KB1 and the remaining 464 are KB2

In [31]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) AS ?kb2Members)
WHERE {
    SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName #MIN(?KBNumber) as ?KB
    WHERE {
        VALUES (?tvSeries) {
            (wd:Q23831)  #'The Office (US)'
            (wd:Q147235) #'How I met your mother'
        }

        ?castMember ^wdt:P161/wdt:P161 ?KB1;       #?KB2 <--wdt:P161 ('cast member')--[]--wdt:P161 ('cast member')-->?KB1
            ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')--> (wd:Q23831)|(wd:Q147235)
            sc:name ?castMemberName.

        ?KB1 ^wdt:P161/wdt:P161 wd:Q3454165;       #?KB1 <--wdt:P161 ('cast member')--[]--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
            ^wdt:P161 ?tvSeries;                   #--wdt:P161 ('cast member')--> (wd:Q23831)|(wd:Q147235)
            sc:name ?KB1Name.

        ?tvSeries wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
            sc:name ?tvSeriesName.

        BIND(IF(?KB1 = ?castMember, 1, 2) AS ?KBNumber) # BINDING KB NUMBER to get the KB2 with HAVING
    }
    GROUP BY ?castMember ?castMemberName ?tvSeries ?tvSeriesName
    HAVING (MIN(?KBNumber) >  1)
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('kb2Members', '464')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('kb2Members', '23')]
2


##### Old sol without PATH PATTERN

In [101]:
# Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2 
#    (the result set must be a list of couples actor/actress IRI and label).

# wdt:P31 ('instance of')--> wd:Q11424 ('film')
#                        wd:Q21191270 ('television series episode')
#                        wd:Q5398426 ('television series')

# wd:Q3454165 ('Kevin Bacon')

# Prob: to much count
# debugging only The Office


queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(*) as ?bacon2CastMembers)
WHERE {      
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE { 
            VALUES (?tvSeries) {
                (wd:Q23831)  #'The Office (US)'
                (wd:Q147235) #'How I met your mother'
            }

            ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
                wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
                sc:name ?tvSeriesName. 

            ?castMember sc:name ?castMemberName.
        }
    }
    MINUS # BACON NUMBER 1
    {
        SELECT DISTINCT ?castMember ?castMemberName ?tvSeries ?tvSeriesName
        WHERE {
           VALUES (?tvSeries) {
               (wd:Q23831)  #'The Office (US)'
               (wd:Q147235) #'How I met your mother'
           }

           ?tvSeries wdt:P161 ?castMember;      #--wdt:P161 ('cast member')-->
               wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
               sc:name ?tvSeriesName. 

           ?castMember sc:name ?castMemberName.    

           ?filmOrSeriesOfKevin wdt:P161 wd:Q3454165;      #--wdt:P161 ('cast member')-->wd:Q3454165 ('Kevin Bacon')
               sc:name ?filmOrSeriesOfKevinName.    
               
           # KBnumber = 1
           ?filmOrSeriesOfKevin wdt:P161 ?castMember.
        }
    }
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('bacon2CastMembers', '464')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23831'), ('tvSeriesName', 'The Office'), ('bacon2CastMembers', '23')]
2


## Task 7
Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

In [102]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')             --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')

# GETTING awards

queryString = """
SELECT DISTINCT ?castMember ?castMemberName
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember;   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
       ?p ?o.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   ?p sc:name ?pName.
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q465556'), ('castMemberName', 'Annie Ilonzeh')]
[('castMember', 'http://www.wikidata.org/entity/Q269891'), ('castMemberName', 'Julianna Guill')]
[('castMember', 'http://www.wikidata.org/entity/Q4888924'), ('castMemberName', 'Benjamin Koldyke')]
[('castMember', 'http://www.wikidata.org/entity/Q446031'), ('castMemberName', 'Nikki Griffin')]
[('castMember', 'http://www.wikidata.org/entity/Q312705'), ('castMemberName', 'John Cho')]
[('castMember', 'http://www.wikidata.org/entity/Q469579'), ('castMemberName', 'Mircea Monroe')]
[('castMember', 'http://www.wikidata.org/entity/Q516659'), ('castMemberName', 'Virginia Williams')]
[('castMember', 'http://www.wikidata.org/entity/Q522856'), ('castMemberName', 'Kate Micucci')]
[('castMember', 'http://www.wikidata.org/entity/Q200566'), ('castMemberName', 'Cobie Smulders')]
[('castMember', 'http://www.wikidata.org/entity/Q435839'), ('castMemberName', 'Ashley Williams')]
10


In [103]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')             --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')

# GETTING awards

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember;   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
       ?p ?o.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   ?p sc:name ?pName.
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
10


In [104]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')

# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember;   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
       wdt:P1411 ?nomination;
       ?p ?o.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   ?p sc:name ?pName.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pName', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('p', 'http://www.wikida

In [105]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?award ?awardName
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember;   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
       wdt:P166 ?award.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   
   OPTIONAL {
       ?award sc:name ?awardName.
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q2300462'), ('awardName', 'Streamy Awards')]
[('award', 'http://www.wikidata.org/entity/Q3002950'), ('awardName', "Critics' Choice Television Award for Best Supporting Actor in a Comedy Series")]
[('award', 'http://www.wikidata.org/entity/Q7243510'), ('awardName', 'Primetime Emmy Award for Outstanding Single-Camera Picture Editing for a Comedy Series')]
[('award', 'http://www.wikidata.org/entity/Q23011211'), ('awardName', 'International TV Audience Award for Best Comedy TV Series')]
[('award', 'http://www.wikidata.org/entity/Q7243501'), ('awardName', 'Primetime Emmy Award for Outstanding Cinematography for a Multi-Camera Series')]
[('award', 'http://www.wikidata.org/entity/Q17985609'), ('awardName', 'Primetime Emmy Award for Outstanding Art Direction for a Contemporary Program')]
[('award', 'http://www.wikidata.org/entity/Q30633707'), ('awardName', 'Primetime Emmy Award for Outstanding Multi-Camera Picture Editing for a Comedy Series')

In [106]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?award ?awardName
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
   
   ?tvSeries wdt:P161 ?castMember;
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       wdt:P166 ?award;
       sc:name ?tvSeriesName.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   
   OPTIONAL {
       ?award sc:name ?awardName.
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q2300462'), ('awardName', 'Streamy Awards')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q3002950'), ('awardName', "Critics' Choice Television Award for Best Supporting Actor in a Comedy Series")]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q7243510'), ('awardName', 'Primetime Emmy Award for Outstanding Single-Camera Picture Editing for a Comedy Series')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q23011211'), ('awardName', 'International TV Audience Award for Best Comedy TV Series')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeri

#### CONSIDERING ALSO Eligible HIMYM, OTHERWISE just add FILTER like (?tvSeries != wd:Q147235)

In [107]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(?award) AS ?awards)
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
   
   ?tvSeries wdt:P161 ?castMember;
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       wdt:P166 ?award;
       sc:name ?tvSeriesName.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   
   OPTIONAL {
       ?award sc:name ?awardName.
   }
   
}
GROUP BY ?tvSeries ?tvSeriesName
HAVING (COUNT(?award) > 2)
ORDER BY DESC(?awards)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('awards', '3360')]
[('tvSeries', 'http://www.wikidata.org/entity/Q34659'), ('tvSeriesName', 'My Name Is Earl'), ('awards', '119')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1079'), ('tvSeriesName', 'Breaking Bad'), ('awards', '105')]
[('tvSeries', 'http://www.wikidata.org/entity/Q16756'), ('tvSeriesName', 'Modern Family'), ('awards', '88')]
[('tvSeries', 'http://www.wikidata.org/entity/Q117396'), ('tvSeriesName', 'CSI: Vegas'), ('awards', '84')]
[('tvSeries', 'http://www.wikidata.org/entity/Q438406'), ('tvSeriesName', "Grey's Anatomy"), ('awards', '80')]
[('tvSeries', 'http://www.wikidata.org/entity/Q8539'), ('tvSeriesName', 'The Big Bang Theory'), ('awards', '60')]
[('tvSeries', 'http://www.wikidata.org/entity/Q19570'), ('tvSeriesName', 'The Good Wife'), ('awards', '39')]
[('tvSeries', 'http://www.wikidata.org/entity/Q244803'), ('tvSeriesName', 'Ally McBeal'), ('awards', '

##### Problem:
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('awards', '3360')]
should have only 7
##### DEBUGGING

In [108]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?award ?awardName
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
   
   ?tvSeries wdt:P161 ?castMember;
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       wdt:P166 ?award;
       sc:name ?tvSeriesName.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   
   OPTIONAL {
       ?award sc:name ?awardName.
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q2300462'), ('awardName', 'Streamy Awards')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q3002950'), ('awardName', "Critics' Choice Television Award for Best Supporting Actor in a Comedy Series")]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q7243510'), ('awardName', 'Primetime Emmy Award for Outstanding Single-Camera Picture Editing for a Comedy Series')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q23011211'), ('awardName', 'International TV Audience Award for Best Comedy TV Series')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeri

In [109]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName ?award ?awardName
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
   
   ?tvSeries wdt:P161 ?castMember;
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       wdt:P166 ?award;
       sc:name ?tvSeriesName.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   
   OPTIONAL {
       ?award sc:name ?awardName.
   }
   
   FILTER (regex(?tvSeriesName, "How I Met"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q2300462'), ('awardName', 'Streamy Awards')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q3002950'), ('awardName', "Critics' Choice Television Award for Best Supporting Actor in a Comedy Series")]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q7243510'), ('awardName', 'Primetime Emmy Award for Outstanding Single-Camera Picture Editing for a Comedy Series')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('award', 'http://www.wikidata.org/entity/Q23011211'), ('awardName', 'International TV Audience Award for Best Comedy TV Series')]
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeri

In [110]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName COUNT(?award) as ?awards
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
   
   ?tvSeries wdt:P161 ?castMember;
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       wdt:P166 ?award;
       sc:name ?tvSeriesName.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   
   OPTIONAL {
       ?award sc:name ?awardName.
   }
   
   FILTER (regex(?tvSeriesName, "How I Met"))
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('awards', '3360')]
1


In [113]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName COUNT(?tvSeriesName) as ?awards
WHERE {
   # bind something
   wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->
   
   ?tvSeries wdt:P161 ?castMember;
       wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
       wdt:P166 ?award;
       sc:name ?tvSeriesName.
              
   # get the label
   ?castMember sc:name ?castMemberName.
   
   OPTIONAL {
       ?award sc:name ?awardName.
   }
   
   FILTER (regex(?tvSeriesName, "How I Met"))
}
GROUP BY ?tvSeries ?tvSeriesName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('awards', '3360')]
1


In [115]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName COUNT(?award) as ?awards
WHERE{
    SELECT DISTINCT ?tvSeries ?tvSeriesName ?award
    WHERE {
       # bind something
       wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->

       ?tvSeries wdt:P161 ?castMember;
           wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
           wdt:P166 ?award;
           sc:name ?tvSeriesName.

       # get the label
       ?castMember sc:name ?castMemberName.

       OPTIONAL {
           ?award sc:name ?awardName.
       }

       FILTER (regex(?tvSeriesName, "How I Met"))
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q147235'), ('tvSeriesName', 'How I Met Your Mother'), ('awards', '7')]
1


##### END DEBUGGING, solved with outer SELECT

In [116]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# BACK TO counting

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName COUNT(?award) as ?awards
WHERE{
    SELECT DISTINCT ?tvSeries ?tvSeriesName ?award
    WHERE {
       # bind something
       wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->

       ?tvSeries wdt:P161 ?castMember;
           wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
           wdt:P166 ?award;
           sc:name ?tvSeriesName.

       # get the label
       ?castMember sc:name ?castMemberName.

       OPTIONAL {
           ?award sc:name ?awardName.
       }
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q1064766'), ('tvSeriesName', 'Mad TV'), ('awards', '1')]
[('tvSeries', 'http://www.wikidata.org/entity/Q464865'), ('tvSeriesName', 'The Nanny'), ('awards', '1')]
[('tvSeries', 'http://www.wikidata.org/entity/Q117590'), ('tvSeriesName', 'Beverly Hills, 90210'), ('awards', '1')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23609'), ('tvSeriesName', 'Weeds'), ('awards', '2')]
[('tvSeries', 'http://www.wikidata.org/entity/Q647001'), ('tvSeriesName', 'Dr. Quinn, Medicine Woman'), ('awards', '2')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1136370'), ('tvSeriesName', 'General Hospital'), ('awards', '10')]
[('tvSeries', 'http://www.wikidata.org/entity/Q16756'), ('tvSeriesName', 'Modern Family'), ('awards', '11')]
[('tvSeries', 'http://www.wikidata.org/entity/Q23594'), ('tvSeriesName', 'Homeland'), ('awards', '2')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1068952'), ('tvSeriesName', 'St. Elsewhere'), ('awards', '8')]
[('tvSeries

In [117]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# BACK TO counting 
# GROUP BY
# + HAVING
# + ORDER BY

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(?award) as ?awards)
WHERE{
    SELECT DISTINCT ?tvSeries ?tvSeriesName ?award
    WHERE {
       # bind something
       wd:Q147235 wdt:P161 ?castMember.   # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->

       ?tvSeries wdt:P161 ?castMember;
           wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
           wdt:P166 ?award;
           sc:name ?tvSeriesName.

       # get the label
       ?castMember sc:name ?castMemberName.

       OPTIONAL {
           ?award sc:name ?awardName.
       }
    }
}
GROUP BY ?tvSeries ?tvSeriesName
HAVING (COUNT(?award) > 2)
ORDER BY DESC(?awards)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23628'), ('tvSeriesName', 'The Sopranos'), ('awards', '22')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1079'), ('tvSeriesName', 'Breaking Bad'), ('awards', '21')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1132439'), ('tvSeriesName', 'The Practice'), ('awards', '12')]
[('tvSeries', 'http://www.wikidata.org/entity/Q16756'), ('tvSeriesName', 'Modern Family'), ('awards', '11')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1136370'), ('tvSeriesName', 'General Hospital'), ('awards', '10')]
[('tvSeries', 'http://www.wikidata.org/entity/Q8539'), ('tvSeriesName', 'The Big Bang Theory'), ('awards', '10')]
[('tvSeries', 'http://www.wikidata.org/entity/Q79784'), ('tvSeriesName', 'Friends'), ('awards', '10')]
[('tvSeries', 'http://www.wikidata.org/entity/Q244803'), ('tvSeriesName', 'Ally McBeal'), ('awards', '9')]
[('tvSeries', 'http://www.wikidata.org/entity/Q438406'), ('tvSeriesName', "Grey's Anatomy"), ('awards', '8')]
[('tvSerie

In [118]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161('cast member')--> wd:Q485310 ('Neil Patrick Harris')
#                                       --wdt:P1411 ('nominated for')
#                                       --wdt:P166 ('award received')
# GETTING awards
# INCREASING LIMIT
# CHECING WITHOUT BINDING TO HIMYM

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(?award) as ?awards)
WHERE{
    SELECT DISTINCT ?tvSeries ?tvSeriesName ?award
    WHERE {
       # bind something
       ?tvSeries wdt:P161 ?castMember;
           wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
           wdt:P166 ?award;
           sc:name ?tvSeriesName.

       # get the label
       ?castMember sc:name ?castMemberName.

       OPTIONAL {
           ?award sc:name ?awardName.
       }
    }
}
GROUP BY ?tvSeries ?tvSeriesName
HAVING (COUNT(?award) > 2)
ORDER BY DESC(?awards)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23628'), ('tvSeriesName', 'The Sopranos'), ('awards', '22')]
[('tvSeries', 'http://www.wikidata.org/entity/Q3577037'), ('tvSeriesName', 'The West Wing'), ('awards', '22')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1079'), ('tvSeriesName', 'Breaking Bad'), ('awards', '21')]
[('tvSeries', 'http://www.wikidata.org/entity/Q56153643'), ('tvSeriesName', 'Watchmen'), ('awards', '20')]
[('tvSeries', 'http://www.wikidata.org/entity/Q192837'), ('tvSeriesName', 'Sherlock'), ('awards', '16')]
[('tvSeries', 'http://www.wikidata.org/entity/Q16868648'), ('tvSeriesName', 'Fleabag'), ('awards', '12')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1132439'), ('tvSeriesName', 'The Practice'), ('awards', '12')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1048856'), ('tvSeriesName', 'All in the Family'), ('awards', '11')]
[('tvSeries', 'http://www.wikidata.org/entity/Q16756'), ('tvSeriesName', 'Modern Family'), ('awards', '11')]
[('tvSeries', 

### Final query for this task

In [119]:
# Consider the actors who are members of the cast of HIMYM. 
#    Amongst the tv series which these actors acted return only those which received more than 2 awards 
#    (the result set must be triples of tv series IRI, label, #awards won).

# wd:Q147235 ('How I met your mother')  --wdt:P161 ('cast member')----->
#                                       --wdt:P1411 ('nominated for')-->
#                                       --wdt:P166 ('award received')-->
# more than 2 =: >2

queryString = """
SELECT DISTINCT ?tvSeries ?tvSeriesName (COUNT(?award) as ?awards)
WHERE{
    SELECT DISTINCT ?tvSeries ?tvSeriesName ?award
    WHERE {
       # bind something
       wd:Q147235 wdt:P161 ?castMember.     # wd:Q147235  ('How I met your mother') --wdt:P161('cast member')-->

       ?tvSeries wdt:P161 ?castMember;      # tv series from the same ?castMember from above
           wdt:P31 wd:Q5398426;             #--wdt:P31 ('instance of')--> wd:Q5398426 ('television series')
           wdt:P166 ?award;                 #--wdt:P166 ('award received')-->
           sc:name ?tvSeriesName.

    }
}
GROUP BY ?tvSeries ?tvSeriesName
HAVING (COUNT(?award) > 2)
ORDER BY DESC(?awards)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tvSeries', 'http://www.wikidata.org/entity/Q23628'), ('tvSeriesName', 'The Sopranos'), ('awards', '22')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1079'), ('tvSeriesName', 'Breaking Bad'), ('awards', '21')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1132439'), ('tvSeriesName', 'The Practice'), ('awards', '12')]
[('tvSeries', 'http://www.wikidata.org/entity/Q16756'), ('tvSeriesName', 'Modern Family'), ('awards', '11')]
[('tvSeries', 'http://www.wikidata.org/entity/Q1136370'), ('tvSeriesName', 'General Hospital'), ('awards', '10')]
[('tvSeries', 'http://www.wikidata.org/entity/Q8539'), ('tvSeriesName', 'The Big Bang Theory'), ('awards', '10')]
[('tvSeries', 'http://www.wikidata.org/entity/Q79784'), ('tvSeriesName', 'Friends'), ('awards', '10')]
[('tvSeries', 'http://www.wikidata.org/entity/Q244803'), ('tvSeriesName', 'Ally McBeal'), ('awards', '9')]
[('tvSeries', 'http://www.wikidata.org/entity/Q438406'), ('tvSeriesName', "Grey's Anatomy"), ('awards', '8')]
[('tvSerie